# "저공해차 보급촉진을 위한 제도운영 지원 및 개선방안 마련" 데이터 분석

## library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from tqdm import tqdm

In [2]:
# 소수점 자리 표기 옵션
pd.options.display.float_format = '{:.5f}'.format
# pd.reset_option('display.float_format')

### data load

In [3]:
# - 2020년
#     - 1월 : 229958
#     - 2월 : 180605
#     - 3월 : 173919
#     - 4월 : 178250
#     - 5월 : 193126
#     - 6월 : 204171
#     - 7월 : 224591
#     - 8월 : 259094
#     - 9월 : 231712
#     - 10월 : 295100
#     - 11월 : 303539
#     - 12월 : 322549
# - 2021년
#     - 1월 : 304612
#     - 2월 : 276148
#     - 3월 : 295915
#     - 4월 : 284649
#     - 5월 : 288367
#     - 6월 : 290878
#     - 7월 : 349382
#     - 8월 : 358950
#     - 9월 : 357788
#     - 10월 : 422421
#     - 11월 : 479016
#     - 12월 : 551719
# - 2022년
#     - 1월 : 544858
#     - 2월 : 465335
#     - 3월 : 512731
#     - 4월 : 544984
#     - 5월 : 593030
#     - 6월 : 597657
#     - 7월 : 675689
#     - 8월 : 675079

In [5]:
total_data = pd.DataFrame()
# 2020년 
count = 0
for m in range(1, 13):
    y = 20
    s = f'raw/{y}년/20{y}년{m:0>2}월.xlsx'
    m_data = pd.read_excel(s)
    print(f'20{y}년 {m:0>2}월 샘플 :', m_data.shape[0])
    count += m_data.shape[0]
    total_data = pd.concat([total_data, m_data], ignore_index=True)
# 2021년
for m in range(1, 13):
    y = 21
    s = f'raw/{y}년/{y}년{m:0>2}월.xlsx'
    m_data = pd.read_excel(s)
    print(f'20{y}년 {m:0>2}월 샘플 :', m_data.shape[0])
    count += m_data.shape[0]
    total_data = pd.concat([total_data, m_data], ignore_index=True)
# 2022년
for m in range(1, 9):
    y = 22
    s = f'raw/{y}년/{y}년{m:0>2}월.xlsx'
    m_data = pd.read_excel(s)
    print(f'20{y}년 {m:0>2}월 샘플 :', m_data.shape[0])
    count += m_data.shape[0]
    total_data = pd.concat([total_data, m_data], ignore_index=True)

2020년 01월 샘플 : 229958
2020년 02월 샘플 : 180605
2020년 03월 샘플 : 173919
2020년 04월 샘플 : 178250
2020년 05월 샘플 : 193126
2020년 06월 샘플 : 204171
2020년 07월 샘플 : 224591
2020년 08월 샘플 : 259094
2020년 09월 샘플 : 231712
2020년 10월 샘플 : 295100
2020년 11월 샘플 : 303539
2020년 12월 샘플 : 322549
2021년 01월 샘플 : 304612
2021년 02월 샘플 : 276148
2021년 03월 샘플 : 295915
2021년 04월 샘플 : 284649
2021년 05월 샘플 : 288367
2021년 06월 샘플 : 290878
2021년 07월 샘플 : 349382
2021년 08월 샘플 : 358950
2021년 09월 샘플 : 357788
2021년 10월 샘플 : 422421
2021년 11월 샘플 : 479016
2021년 12월 샘플 : 551719
2022년 01월 샘플 : 544858
2022년 02월 샘플 : 465335
2022년 03월 샘플 : 512731
2022년 04월 샘플 : 544984
2022년 05월 샘플 : 593030
2022년 06월 샘플 : 597657
2022년 07월 샘플 : 675689
2022년 08월 샘플 : 675079


In [6]:
print(f'개별 데이터 개수 합산 : {count}')
print(f'총 데이터 개수 : {total_data.shape[0]}')

개별 데이터 개수 합산 : 11665822
총 데이터 개수 : 11665822


In [7]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665822 entries, 0 to 11665821
Data columns (total 12 columns):
 #   Column     Dtype  
---  ------     -----  
 0   충전소명       object 
 1   충전기ID      int64  
 2   지역         object 
 3   시군구        object 
 4   주소         object 
 5   충전기용량(kW)  float64
 6   충전기타입      object 
 7   충전시작일시     int64  
 8   충전종료일시     int64  
 9   충전시간       object 
 10  충전량        float64
 11  충전용량       object 
dtypes: float64(2), int64(3), object(7)
memory usage: 1.0+ GB


#### 데이터 절약

In [8]:
total_data['충전기ID'] = total_data['충전기ID'].astype('int16')
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665822 entries, 0 to 11665821
Data columns (total 12 columns):
 #   Column     Dtype  
---  ------     -----  
 0   충전소명       object 
 1   충전기ID      int16  
 2   지역         object 
 3   시군구        object 
 4   주소         object 
 5   충전기용량(kW)  float64
 6   충전기타입      object 
 7   충전시작일시     int64  
 8   충전종료일시     int64  
 9   충전시간       object 
 10  충전량        float64
 11  충전용량       object 
dtypes: float64(2), int16(1), int64(2), object(7)
memory usage: 1001.3+ MB


##### 20-22년 병합 데이터 출력

In [80]:
total_data.to_csv('2020-22년_급속충전.csv', encoding='utf-8')

##### 20-22년 병합 데이터 로드(다시 시작)

In [638]:
# total_data = pd.read_csv('2020-22년_급속충전.csv', index_col=0)
# total_data.info()

C:\Users\kbj\AppData\Local\Temp\ipykernel_17248\1573190630.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  total_data = pd.read_csv('2020-22년_급속충전.csv')


In [9]:
df = total_data.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665822 entries, 0 to 11665821
Data columns (total 12 columns):
 #   Column     Dtype  
---  ------     -----  
 0   충전소명       object 
 1   충전기ID      int16  
 2   지역         object 
 3   시군구        object 
 4   주소         object 
 5   충전기용량(kW)  float64
 6   충전기타입      object 
 7   충전시작일시     int64  
 8   충전종료일시     int64  
 9   충전시간       object 
 10  충전량        float64
 11  충전용량       object 
dtypes: float64(2), int16(1), int64(2), object(7)
memory usage: 1001.3+ MB


## 전처리
- 충전소명 : O
- 충전기ID : int64
- 지역 : O
- 시군구 : O
- 주소 : O
- 충전기용량(kW) : float64
- 충전기타입 : object 
- 충전시작일시 : int64  
- 충전종료일시 : int64  
- 충전시간 : object 
- 충전량 : float64
- 충전용량 : object 

### 특이사항
- 2020년 12월까지 충전기용량(kW)가 2021년 1월부터 충전용량으로 변경
    - 2020년 12월까지 데이터의 충전기용량(kW) 값을 충전용량으로 이동
- 충전용량 표기방법 변경
    - 2022년 3월까지 충전용량이 "100"으로 , 4월부터 "급속(100kW멀티)"으로 입력

In [10]:
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.00000,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47000,NaN
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.00000,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57000,NaN
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.00000,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98000,NaN
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.00000,DC콤보,20200120120215,20200120124708,00:41:00,15.30000,NaN
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.00000,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92000,NaN


In [11]:
df[df['충전용량'].notnull() == True].head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량
2796614,군위휴게소(부산방향),1,경상북도,군위군,경상북도 군위군 군위읍 경북대로 4084 (오곡리),NaN,DC차데모+AC3상+DC콤보,20210101092250,20210101095816,00:29:24,11.95000,50.00000
2796615,충주휴게소(마산방향),1,충청북도,충주시,충청북도 충주시 중앙탑면 중부내륙고속도로 231 (용전리),NaN,DC차데모+AC3상+DC콤보,20210101095126,20210101095818,00:06:48,4.86000,50.00000
2796616,내서읍스포츠센터,1,경상남도,창원시,경상남도 창원시 마산회원구 내서읍 중리 391-3,NaN,DC콤보,20210101094752,20210101095819,00:10:27,3.29000,100.00000
2796617,삼성혈 인근 공영주차장,1,제주특별자치도,제주시,제주특별자치도 제주시 이도1동 1293-1,NaN,DC차데모+AC3상+DC콤보,20210101091323,20210101095819,00:40:00,7.52000,50.00000
2796618,본오1동행정복지센터,1,경기도,안산시,경기도 안산시 상록구 샘골로 16,NaN,DC콤보,20210101091559,20210101095835,00:40:00,21.66000,100.00000


### 충전시작시간 정리

In [12]:
# 충전시작일시의 데이터는 년~초 까지 모두 존재
temp = df['충전시작일시'].to_list()
temp2 = [str(i) for i in temp]
count = 0
for one in temp2:
    if len(one) == 8:
        count += 1
print(count)

0


In [13]:
ch_start = df['충전시작일시'].copy()
df['충전시작_년도'] = ch_start.astype(str).str[:4].astype(int)
df['충전시작_월'] = df['충전시작일시'].astype(str).str[4:6].astype(int)
df['충전시작_일'] = df['충전시작일시'].astype(str).str[6:8].astype(int)
df['충전시작_시'] = df['충전시작일시'].astype(str).str[8:10].astype(int)
df['충전시작_분'] = df['충전시작일시'].astype(str).str[10:12].astype(int)
df['충전시작_초'] = df['충전시작일시'].astype(str).str[12:14].astype(int)
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.00000,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47000,NaN,2020,1,17,19,33,31
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.00000,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57000,NaN,2020,1,30,14,35,45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.00000,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98000,NaN,2020,1,5,0,57,28
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.00000,DC콤보,20200120120215,20200120124708,00:41:00,15.30000,NaN,2020,1,20,12,2,15
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.00000,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92000,NaN,2020,1,23,8,54,17


In [14]:
df['충전시작일시(변환)'] = df['충전시작_년도'].astype(str) + '-' + df['충전시작_월'].astype(str) + '-' + df['충전시작_일'].astype(str) + ' ' + df['충전시작_시'].astype(str) + ':' + df['충전시작_분'].astype(str) + ':' + df['충전시작_초'].astype(str)
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.00000,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47000,NaN,2020,1,17,19,33,31,2020-1-17 19:33:31
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.00000,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57000,NaN,2020,1,30,14,35,45,2020-1-30 14:35:45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.00000,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98000,NaN,2020,1,5,0,57,28,2020-1-5 0:57:28
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.00000,DC콤보,20200120120215,20200120124708,00:41:00,15.30000,NaN,2020,1,20,12,2,15,2020-1-20 12:2:15
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.00000,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92000,NaN,2020,1,23,8,54,17,2020-1-23 8:54:17


In [15]:
df['충전시작일시(변환)'] = pd.to_datetime(df['충전시작일시(변환)'])
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.00000,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47000,NaN,2020,1,17,19,33,31,2020-01-17 19:33:31
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.00000,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57000,NaN,2020,1,30,14,35,45,2020-01-30 14:35:45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.00000,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98000,NaN,2020,1,5,0,57,28,2020-01-05 00:57:28
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.00000,DC콤보,20200120120215,20200120124708,00:41:00,15.30000,NaN,2020,1,20,12,2,15,2020-01-20 12:02:15
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.00000,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92000,NaN,2020,1,23,8,54,17,2020-01-23 08:54:17


### 충전종료시간 정리

In [16]:
df[df['충전종료일시'] == ' ']

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환)


In [17]:
# 충전종료일시의 데이터는 년~일까지만 있는 데이터가 71311개 존재
    # 해당 데이터 뒤에 시분초를 '000000'으로 추가하는 작업 필요함.
temp = df['충전종료일시'].to_list()
temp2 = [str(i) for i in temp]
count = 0
for one in temp2:
    if len(one) == 8:
        count += 1
print(count)

71311


In [18]:
df.tail()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환)
11665817,죽전휴게소(서울방향),1,경기도,용인시,경기도 용인시 수지구 경부고속도로 400 (죽전동),NaN,DC차데모+AC3상+DC콤보,20220831235918,20220901004034,00:40:00,31.09000,급속(50kW),2022,8,31,23,59,18,2022-08-31 23:59:18
11665818,원주모범운전자회,21,강원도,원주시,강원도 원주시 단구로 150,NaN,DC콤보,20220831235920,20220901001904,00:19:43,46.51000,급속(200kW동시),2022,8,31,23,59,20,2022-08-31 23:59:20
11665819,검단2공영주차장,1,인천광역시,서구,인천광역시 서구 당하동 1097-5 검단4동 주민센터 옆,NaN,DC차데모+AC3상+DC콤보,20220831235931,20220901004251,00:40:00,21.97000,급속(50kW),2022,8,31,23,59,31,2022-08-31 23:59:31
11665820,강당골 공영주차장,1,대구광역시,남구,대구광역시 남구 봉덕동 1272-18,NaN,DC차데모+AC3상+DC콤보,20220831235934,20220901004057,00:40:00,27.60000,급속(50kW),2022,8,31,23,59,34,2022-08-31 23:59:34
11665821,마천동2공영주차장,1,서울특별시,송파구,서울특별시 송파구 성내천로299-7,NaN,DC차데모+AC3상+DC콤보,20220831235946,20220901003700,00:34:09,9.77000,급속(100kW멀티),2022,8,31,23,59,46,2022-08-31 23:59:46


In [19]:
df['충전종료일시'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 11665822 entries, 0 to 11665821
Series name: 충전종료일시
Non-Null Count     Dtype
--------------     -----
11665822 non-null  int64
dtypes: int64(1)
memory usage: 89.0 MB


In [20]:
temp = df['충전종료일시'].to_list()

In [21]:
for i, one in enumerate(temp):
    if len(str(one)) == 8:
        print(i)
        break

1578


In [22]:
temp[1578]

20200118

In [23]:
df['충전종료일시'][1578]

20200118

In [24]:
temp2 = [str(i) for i in temp]
temp2[1578]

'20200118'

In [25]:
for i, one in enumerate(temp2):
    if len(one) == 8:
        temp2[i] = one + '000000'
len(temp2)

11665822

In [26]:
temp2[1578]

'20200118000000'

In [27]:
df['충전종료일시'] = temp2
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.00000,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47000,NaN,2020,1,17,19,33,31,2020-01-17 19:33:31
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.00000,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57000,NaN,2020,1,30,14,35,45,2020-01-30 14:35:45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.00000,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98000,NaN,2020,1,5,0,57,28,2020-01-05 00:57:28
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.00000,DC콤보,20200120120215,20200120124708,00:41:00,15.30000,NaN,2020,1,20,12,2,15,2020-01-20 12:02:15
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.00000,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92000,NaN,2020,1,23,8,54,17,2020-01-23 08:54:17


In [30]:
df['충전종료일시'][1578]

'20200118000000'

In [29]:
df.iloc[1578, :]

충전소명                동해웰빙레포츠타운 주차장
충전기ID                          21
지역                            강원도
시군구                           동해시
주소                 강원도 동해시 덕골길 10
충전기용량(kW)               200.00000
충전기타입                        DC콤보
충전시작일시             20200118162253
충전종료일시             20200118000000
충전시간                     00:06:49
충전량                       1.91000
충전용량                          NaN
충전시작_년도                      2020
충전시작_월                          1
충전시작_일                         18
충전시작_시                         16
충전시작_분                         22
충전시작_초                         53
충전시작일시(변환)    2020-01-18 16:22:53
Name: 1578, dtype: object

In [31]:
ch_time = df['충전종료일시'].copy()
df['충전종료_년도'] = ch_time.astype(str).str[:4].astype(int)
df['충전종료_월'] = ch_time.astype(str).str[4:6].astype(int)
df['충전종료_일'] = ch_time.astype(str).str[6:8].astype(int)
df['충전종료_시'] = ch_time.astype(str).str[8:10].astype(int)
df['충전종료_분'] = ch_time.astype(str).str[10:12].astype(int)
df['충전종료_초'] = ch_time.astype(str).str[12:14].astype(int)
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.00000,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,...,19,33,31,2020-01-17 19:33:31,2020,1,17,20,14,17
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.00000,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,...,14,35,45,2020-01-30 14:35:45,2020,1,30,15,6,45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.00000,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,...,0,57,28,2020-01-05 00:57:28,2020,1,5,1,13,29
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.00000,DC콤보,20200120120215,20200120124708,00:41:00,...,12,2,15,2020-01-20 12:02:15,2020,1,20,12,47,8
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.00000,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,...,8,54,17,2020-01-23 08:54:17,2020,1,23,9,25,54


In [32]:
df.tail()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초
11665817,죽전휴게소(서울방향),1,경기도,용인시,경기도 용인시 수지구 경부고속도로 400 (죽전동),NaN,DC차데모+AC3상+DC콤보,20220831235918,20220901004034,00:40:00,...,23,59,18,2022-08-31 23:59:18,2022,9,1,0,40,34
11665818,원주모범운전자회,21,강원도,원주시,강원도 원주시 단구로 150,NaN,DC콤보,20220831235920,20220901001904,00:19:43,...,23,59,20,2022-08-31 23:59:20,2022,9,1,0,19,4
11665819,검단2공영주차장,1,인천광역시,서구,인천광역시 서구 당하동 1097-5 검단4동 주민센터 옆,NaN,DC차데모+AC3상+DC콤보,20220831235931,20220901004251,00:40:00,...,23,59,31,2022-08-31 23:59:31,2022,9,1,0,42,51
11665820,강당골 공영주차장,1,대구광역시,남구,대구광역시 남구 봉덕동 1272-18,NaN,DC차데모+AC3상+DC콤보,20220831235934,20220901004057,00:40:00,...,23,59,34,2022-08-31 23:59:34,2022,9,1,0,40,57
11665821,마천동2공영주차장,1,서울특별시,송파구,서울특별시 송파구 성내천로299-7,NaN,DC차데모+AC3상+DC콤보,20220831235946,20220901003700,00:34:09,...,23,59,46,2022-08-31 23:59:46,2022,9,1,0,37,0


#### 충전종료일시-초 오류 데이터
- 1495개

In [33]:
df[df['충전종료_초'] >= 60].shape

(1495, 25)

In [34]:
over_s_idx = df[df['충전종료_초'] >= 60].index
len(over_s_idx)

1495

In [35]:
over_s_idx[:5]

Int64Index([1960560, 1964993, 1966556, 1976775, 2043371], dtype='int64')

In [38]:
df[['충전종료_분', '충전종료_초']].iloc[1960560]

충전종료_분    30
충전종료_초    99
Name: 1960560, dtype: int32

In [39]:
df.loc[1960560, '충전종료_초']

99

In [40]:
for i in over_s_idx:
    df.loc[i, '충전종료_분'] += 1
    df.loc[i, '충전종료_초'] -= 60

In [42]:
df[['충전종료_분', '충전종료_초']].iloc[1960560]

충전종료_분    31
충전종료_초    39
Name: 1960560, dtype: int32

#### 충전종료일시-분 오류 데이터
- 3270개

In [43]:
df[df['충전종료_분'] >= 60].shape

(3270, 25)

In [44]:
over_m_idx = df[df['충전종료_분'] >= 60].index
len(over_m_idx)

3270

In [45]:
over_m_idx[:5]

Int64Index([1964993, 1966556, 1968601, 1968849, 1970888], dtype='int64')

In [46]:
df[['충전종료_시', '충전종료_분']].iloc[1964993]

충전종료_시    17
충전종료_분    65
Name: 1964993, dtype: int32

In [48]:
df.loc[1964993, '충전종료_분']

65

In [49]:
for i in over_m_idx:
    df.loc[i, '충전종료_시'] += 1
    df.loc[i, '충전종료_분'] -= 60

In [50]:
df.loc[1964993, '충전종료_분']

5

In [51]:
df[['충전종료_시', '충전종료_분']].iloc[1964993]

충전종료_시    18
충전종료_분     5
Name: 1964993, dtype: int32

#### 충전종료일시-시 오류 데이터
- 71개

In [52]:
df[df['충전종료_시'] >= 24].shape

(71, 25)

In [53]:
over_h_idx = df[df['충전종료_시'] >= 24].index
len(over_h_idx)

71

In [54]:
over_h_idx[:5]

Int64Index([2168170, 2171801, 2189569, 2189571, 2192634], dtype='int64')

In [56]:
df[['충전종료_일', '충전종료_시']].iloc[2168170]

충전종료_일    30
충전종료_시    24
Name: 2168170, dtype: int32

In [57]:
for i in over_h_idx:
    df.loc[i, '충전종료_일'] += 1
    df.loc[i, '충전종료_시'] -= 24

In [58]:
df[['충전종료_일', '충전종료_시']].iloc[2168170]

충전종료_일    31
충전종료_시     0
Name: 2168170, dtype: int32

#### 충전종료일시-일 오류 데이터
- 3개(index) : 3232880, 2440150, 2775050

In [59]:
df[(df['충전종료_월'] == 1) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [60]:
df[(df['충전종료_년도'] == 2021) & (df['충전종료_월'] == 2) & (df['충전종료_일'] > 28)].index

Int64Index([3232880], dtype='int64')

In [61]:
df[(df['충전종료_년도'] == 2022) & (df['충전종료_월'] == 2) & (df['충전종료_일'] > 28)].index

Int64Index([], dtype='int64')

In [62]:
# 2020년은 윤년
df[(df['충전종료_년도'] == 2020) & (df['충전종료_월'] == 2) & (df['충전종료_일'] > 29)].index

Int64Index([], dtype='int64')

In [63]:
df[(df['충전종료_월'] == 3) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [64]:
df[(df['충전종료_월'] == 4) & (df['충전종료_일'] > 30)].index

Int64Index([], dtype='int64')

In [65]:
df[(df['충전종료_월'] == 5) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [66]:
df[(df['충전종료_월'] == 6) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [67]:
df[(df['충전종료_월'] == 7) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [68]:
df[(df['충전종료_월'] == 8) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [69]:
df[(df['충전종료_월'] == 9) & (df['충전종료_일'] > 30)].index

Int64Index([], dtype='int64')

In [70]:
df[(df['충전종료_월'] == 10) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [71]:
df[(df['충전종료_월'] == 11) & (df['충전종료_일'] > 30)].index

Int64Index([2440150], dtype='int64')

In [72]:
df[(df['충전종료_월'] == 12) & (df['충전종료_일'] > 31)].index

Int64Index([2775050], dtype='int64')

In [73]:
check_col = ['충전종료_년도', '충전종료_월', '충전종료_일']
df.loc[3232880, check_col]

충전종료_년도    2021
충전종료_월        2
충전종료_일       29
Name: 3232880, dtype: object

In [74]:
df.loc[3232880, '충전종료_월'] = 3
df.loc[3232880, '충전종료_일'] = 1
df.loc[3232880, check_col]

충전종료_년도    2021
충전종료_월        3
충전종료_일        1
Name: 3232880, dtype: object

In [75]:
df.loc[2440150, check_col]

충전종료_년도    2020
충전종료_월       11
충전종료_일       31
Name: 2440150, dtype: object

In [76]:
df.loc[2440150, '충전종료_월'] = 12
df.loc[2440150, '충전종료_일'] = 1
df.loc[2440150, check_col]

충전종료_년도    2020
충전종료_월       12
충전종료_일        1
Name: 2440150, dtype: object

In [77]:
df.loc[2775050, check_col]

충전종료_년도    2020
충전종료_월       12
충전종료_일       32
Name: 2775050, dtype: object

In [78]:
df.loc[2775050, '충전종료_년도'] = 2021
df.loc[2775050, '충전종료_월'] = 1
df.loc[2775050, '충전종료_일'] = 1
df.loc[2775050, check_col]

충전종료_년도    2021
충전종료_월        1
충전종료_일        1
Name: 2775050, dtype: object

#### 충전종료시간(변환)

In [80]:
df['충전종료일시(변환)'] = df['충전종료_년도'].astype(str) + '-' +  df['충전종료_월'].astype(str) + '-' + df['충전종료_일'].astype(str) + ' ' + df['충전종료_시'].astype(str) + ':' + df['충전종료_분'].astype(str) + ':' + df['충전종료_초'].astype(str)
df.iloc[:5,8:]

,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환)
0,20200117201417,00:40:00,25.47000,NaN,2020,1,17,19,33,31,2020-01-17 19:33:31,2020,1,17,20,14,17,2020-1-17 20:14:17
1,20200130150645,00:30:31,17.57000,NaN,2020,1,30,14,35,45,2020-01-30 14:35:45,2020,1,30,15,6,45,2020-1-30 15:6:45
2,20200105011329,00:15:35,6.98000,NaN,2020,1,5,0,57,28,2020-01-05 00:57:28,2020,1,5,1,13,29,2020-1-5 1:13:29
3,20200120124708,00:41:00,15.30000,NaN,2020,1,20,12,2,15,2020-01-20 12:02:15,2020,1,20,12,47,8,2020-1-20 12:47:8
4,20200123092554,00:31:14,21.92000,NaN,2020,1,23,8,54,17,2020-01-23 08:54:17,2020,1,23,9,25,54,2020-1-23 9:25:54


In [81]:
df['충전종료일시(변환)'] = pd.to_datetime(df['충전종료일시(변환)'])

In [82]:
df['충전시간(계산)'] = df['충전종료일시(변환)'] - df['충전시작일시(변환)']
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.00000,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,...,31,2020-01-17 19:33:31,2020,1,17,20,14,17,2020-01-17 20:14:17,0 days 00:40:46
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.00000,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,...,45,2020-01-30 14:35:45,2020,1,30,15,6,45,2020-01-30 15:06:45,0 days 00:31:00
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.00000,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,...,28,2020-01-05 00:57:28,2020,1,5,1,13,29,2020-01-05 01:13:29,0 days 00:16:01
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.00000,DC콤보,20200120120215,20200120124708,00:41:00,...,15,2020-01-20 12:02:15,2020,1,20,12,47,8,2020-01-20 12:47:08,0 days 00:44:53
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.00000,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,...,17,2020-01-23 08:54:17,2020,1,23,9,25,54,2020-01-23 09:25:54,0 days 00:31:37


### 충전기용량과 충전용량 합치기

In [95]:
df = df.sort_values(by=['충전시작_년도', '충전시작_월', '충전시작_일', '충전시작_시', '충전시작_분', '충전시작_초']).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665822 entries, 0 to 11665821
Data columns (total 27 columns):
 #   Column      Dtype          
---  ------      -----          
 0   충전소명        object         
 1   충전기ID       int16          
 2   지역          object         
 3   시군구         object         
 4   주소          object         
 5   충전기용량(kW)   float64        
 6   충전기타입       object         
 7   충전시작일시      int64          
 8   충전종료일시      object         
 9   충전시간        object         
 10  충전량         float64        
 11  충전용량        object         
 12  충전시작_년도     int32          
 13  충전시작_월      int32          
 14  충전시작_일      int32          
 15  충전시작_시      int32          
 16  충전시작_분      int32          
 17  충전시작_초      int32          
 18  충전시작일시(변환)  datetime64[ns] 
 19  충전종료_년도     int32          
 20  충전종료_월      int32          
 21  충전종료_일      int32          
 22  충전종료_시      int32          
 23  충전종료_분      int32          
 24  충전종료_초      int32     

In [96]:
df[df['충전시작_년도'] == 2020].tail()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산)
2796609,시흥제4동공영주차장,1,서울특별시,금천구,"서울특별시 금천구 독산로 36나길 21, B1 18",50.00000,DC차데모+AC3상+DC콤보,20201231235808,20210101002023,00:21:51,...,8,2020-12-31 23:58:08,2021,1,1,0,20,23,2021-01-01 00:20:23,0 days 00:22:15
2796610,건천휴게소(부산방향),1,경상북도,경주시,경상북도 경주시 건천읍 경부고속도로 77 (방내리),50.00000,DC차데모+AC3상+DC콤보,20201231235811,20210101003023,00:31:50,...,11,2020-12-31 23:58:11,2021,1,1,0,30,23,2021-01-01 00:30:23,0 days 00:32:12
2796611,중부지방고용노동청 별관 앞,1,인천광역시,남동구,인천광역시 남동구 구월3동 1111-6 중부지방고용노동청 별관 앞,50.00000,DC차데모+AC3상+DC콤보,20201231235836,20210101001905,00:20:02,...,36,2020-12-31 23:58:36,2021,1,1,0,19,5,2021-01-01 00:19:05,0 days 00:20:29
2796612,세종중앙공원주차장2,2,세종특별자치시,연기면,세종특별자치시 연기면 수목원로 136,100.00000,DC콤보,20201231235931,20210101003714,00:37:43,...,31,2020-12-31 23:59:31,2021,1,1,0,37,14,2021-01-01 00:37:14,0 days 00:37:43
2796613,부석면주민자치센터,1,충청남도,서산시,충청남도 서산시 부석면 취평리 364-2,100.00000,DC차데모+AC3상+DC콤보,20201231235955,20210101001316,00:12:21,...,55,2020-12-31 23:59:55,2021,1,1,0,13,16,2021-01-01 00:13:16,0 days 00:13:21


In [97]:
df[df['충전시작_년도'] == 2021].head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산)
2796614,칠곡3지구 공영주차장,42,대구광역시,북구,대구광역시 북구 구암동 771-2,NaN,DC콤보,20210101000032,20210101002547,00:25:16,...,32,2021-01-01 00:00:32,2021,1,1,0,25,47,2021-01-01 00:25:47,0 days 00:25:15
2796615,망향휴게소(부산방향),1,충청남도,천안시,충청남도 천안시 서북구 성거읍 돌다리길 23-37,NaN,DC차데모+AC3상+DC콤보,20210101000034,20210101001118,00:09:51,...,34,2021-01-01 00:00:34,2021,1,1,0,11,18,2021-01-01 00:11:18,0 days 00:10:44
2796616,관악역 3환승 공영주차장,1,경기도,안양시,경기도 안양시 만안구 석수1동 110-26 입구 맨끝 왼쪽,NaN,DC차데모+AC3상+DC콤보,20210101000043,20210101002454,00:23:47,...,43,2021-01-01 00:00:43,2021,1,1,0,24,54,2021-01-01 00:24:54,0 days 00:24:11
2796617,평창휴게소(강릉방향),1,강원도,평창군,강원도 평창군 용평면 경강로 2159-58,NaN,DC차데모+AC3상+DC콤보,20210101000049,20210101004354,00:40:00,...,49,2021-01-01 00:00:49,2021,1,1,0,43,54,2021-01-01 00:43:54,0 days 00:43:05
2796618,강동구청,12,서울특별시,강동구,서울특별시 강동구 성내로 25,NaN,DC콤보,20210101000052,20210101003015,00:29:03,...,52,2021-01-01 00:00:52,2021,1,1,0,30,15,2021-01-01 00:30:15,0 days 00:29:23


In [98]:
df.loc[df['충전시작_년도'] == 2020, '충전기용량(kW)'].shape

(2796614,)

In [99]:
df.loc[df['충전시작_년도'] == 2020, '충전기용량(kW)'].head()

0   50.00000
1   50.00000
2   50.00000
3   50.00000
4   50.00000
Name: 충전기용량(kW), dtype: float64

In [100]:
df.loc[df['충전시작_년도'] == 2020, '충전기용량(kW)'].tail()

2796609    50.00000
2796610    50.00000
2796611    50.00000
2796612   100.00000
2796613   100.00000
Name: 충전기용량(kW), dtype: float64

In [101]:
temp = df.loc[df['충전시작_년도'] == 2020, '충전기용량(kW)'].to_list()
temp[:5]

[50.0, 50.0, 50.0, 50.0, 50.0]

In [102]:
temp[-5:]

[50.0, 50.0, 50.0, 100.0, 100.0]

In [103]:
len(temp)

2796614

In [114]:
df.loc[df['충전시작_년도'] == 2021, '충전용량'].shape

(4259845,)

In [109]:
df.loc[df['충전시작_년도'] == 2021, '충전용량'].head()

2796614   100.00000
2796615    50.00000
2796616    50.00000
2796617   100.00000
2796618   200.00000
Name: 충전용량, dtype: object

In [110]:
df.loc[df['충전시작_년도'] == 2021, '충전용량'].tail()

7056454   100.00000
7056455    50.00000
7056456   200.00000
7056457   100.00000
7056458    50.00000
Name: 충전용량, dtype: object

In [111]:
temp2 = df.loc[df['충전시작_년도'] == 2021, '충전용량'].to_list()
temp2[:5]

[100.0, 50.0, 50.0, 100.0, 200.0]

In [112]:
temp2[-5:]

[100.0, 50.0, 200.0, 100.0, 50.0]

In [113]:
len(temp2)

4259845

In [115]:
df.loc[df['충전시작_년도'] == 2022, '충전용량'].shape

(4609363,)

In [116]:
df.loc[df['충전시작_년도'] == 2022, '충전용량'].head()

7056459    50.00000
7056460    50.00000
7056461    50.00000
7056462   200.00000
7056463    50.00000
Name: 충전용량, dtype: object

In [120]:
df.loc[df['충전시작_년도'] == 2022, '충전용량'].tail()

11665817       급속(50kW)
11665818    급속(200kW동시)
11665819       급속(50kW)
11665820       급속(50kW)
11665821    급속(100kW멀티)
Name: 충전용량, dtype: object

In [118]:
temp3 = df.loc[df['충전시작_년도'] == 2022, '충전용량'].to_list()
temp3[:5]

[50.0, 50.0, 50.0, 200.0, 50.0]

In [119]:
temp3[-5:]

['급속(50kW)', '급속(200kW동시)', '급속(50kW)', '급속(50kW)', '급속(100kW멀티)']

In [122]:
len(temp3)

4609363

In [121]:
df.shape

(11665822, 27)

In [123]:
len(temp + temp2 + temp3)

11665822

In [ ]:
df['충전용량'].head()

In [124]:
df['충전용량'] = temp + temp2 + temp3

In [127]:
df['충전용량'].head()

0   50.00000
1   50.00000
2   50.00000
3   50.00000
4   50.00000
Name: 충전용량, dtype: object

In [125]:
df['충전기용량(kW)'].tail()

11665817   NaN
11665818   NaN
11665819   NaN
11665820   NaN
11665821   NaN
Name: 충전기용량(kW), dtype: float64

In [128]:
df.iloc[:5, :12]

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량
0,진영휴게소(부산방향),1,경상남도,김해시,"경상남도 김해시 진영읍 하계로96번길 94-4 , 주유소 옆 주차장",50.00000,DC차데모+AC3상+DC콤보,20200101000244,20200101004215,00:39:25,20.22000,50.00000
1,천왕역 환승주차장,1,서울특별시,구로구,"서울특별시 구로구 오류동 174-10 , 천왕역 환승주차장 1층",50.00000,DC차데모+AC3상,20200101000457,20200101004534,00:40:37,18.87000,50.00000
2,민속자연사박물관,1,제주특별자치도,제주시,제주특별자치도 제주시 삼성로 40 민속자연사박물관,50.00000,DC차데모+AC3상+DC콤보,20200101000745,20200101003703,00:29:13,16.04000,50.00000
3,노형동주민센터,1,제주특별자치도,제주시,"제주특별자치도 제주시 노형9길 9-4, 주차장",50.00000,DC차데모+AC3상+DC콤보,20200101000748,20200101001659,00:09:07,5.80000,50.00000
4,영일대1주차장,1,경상북도,포항시,경상북도 포항시 북구 해안로82,50.00000,DC차데모+AC3상+DC콤보,20200101000851,20200101004220,00:33:29,13.32000,50.00000


In [129]:
df = df.drop(['충전기용량(kW)'], axis=1)
df.iloc[:5, :12]

,충전소명,충전기ID,지역,시군구,주소,충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도
0,진영휴게소(부산방향),1,경상남도,김해시,"경상남도 김해시 진영읍 하계로96번길 94-4 , 주유소 옆 주차장",DC차데모+AC3상+DC콤보,20200101000244,20200101004215,00:39:25,20.22000,50.00000,2020
1,천왕역 환승주차장,1,서울특별시,구로구,"서울특별시 구로구 오류동 174-10 , 천왕역 환승주차장 1층",DC차데모+AC3상,20200101000457,20200101004534,00:40:37,18.87000,50.00000,2020
2,민속자연사박물관,1,제주특별자치도,제주시,제주특별자치도 제주시 삼성로 40 민속자연사박물관,DC차데모+AC3상+DC콤보,20200101000745,20200101003703,00:29:13,16.04000,50.00000,2020
3,노형동주민센터,1,제주특별자치도,제주시,"제주특별자치도 제주시 노형9길 9-4, 주차장",DC차데모+AC3상+DC콤보,20200101000748,20200101001659,00:09:07,5.80000,50.00000,2020
4,영일대1주차장,1,경상북도,포항시,경상북도 포항시 북구 해안로82,DC차데모+AC3상+DC콤보,20200101000851,20200101004220,00:33:29,13.32000,50.00000,2020


##### 일시 분리 데이터 출력

In [130]:
df.to_csv('2020-22년_급속충전_일시분리.csv', encoding='utf-8')

## 일시 분리 데이터 로드(다시 시작)

In [5]:
# df = pd.read_csv('2020-22년_급속충전_일시분리.csv', index_col=0)
# df.info()

C:\Users\kbj\AppData\Local\Temp\ipykernel_7780\927583139.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('2020-22년_급속충전_일시분리.csv')


In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665822 entries, 0 to 11665821
Data columns (total 26 columns):
 #   Column      Dtype          
---  ------      -----          
 0   충전소명        object         
 1   충전기ID       int16          
 2   지역          object         
 3   시군구         object         
 4   주소          object         
 5   충전기타입       object         
 6   충전시작일시      int64          
 7   충전종료일시      object         
 8   충전시간        object         
 9   충전량         float64        
 10  충전용량        object         
 11  충전시작_년도     int32          
 12  충전시작_월      int32          
 13  충전시작_일      int32          
 14  충전시작_시      int32          
 15  충전시작_분      int32          
 16  충전시작_초      int32          
 17  충전시작일시(변환)  datetime64[ns] 
 18  충전종료_년도     int32          
 19  충전종료_월      int32          
 20  충전종료_일      int32          
 21  충전종료_시      int32          
 22  충전종료_분      int32          
 23  충전종료_초      int32          
 24  충전종료일시(변환)  datetime64

### raw 충전시간과 계산값 비교

In [132]:
df['충전시간'].head()

0    00:39:25
1    00:40:37
2    00:29:13
3    00:09:07
4    00:33:29
Name: 충전시간, dtype: object

In [133]:
df['충전시간'].dtype

dtype('O')

#### 충전시간 오류 값 처리
- 7개 제거

In [139]:
df.loc[[1165409, 1191927, 1196581, 1196582, 7030914, 7305076, 11392158], '충전시간']

1165409     -6:-2:-5
1191927     00:-4:-1
1196581     00:-1:-4
1196582     00:-1:-2
7030914     00:-4:-2
7305076     00:-1:-3
11392158    -1:-4:-4
Name: 충전시간, dtype: object

In [135]:
# temp = pd.to_timedelta(df['충전시간']) # only leading negative signs are allowed 에러 발생
temp = pd.to_timedelta(df['충전시간'], errors='coerce')

In [136]:
temp.isnull().sum()

7

In [137]:
temp = temp.to_frame()

In [138]:
temp[temp['충전시간'].isnull() == True]

,충전시간
1165409,NaT
1191927,NaT
1196581,NaT
1196582,NaT
7030914,NaT
7305076,NaT
11392158,NaT


In [140]:
null_idx = temp[temp['충전시간'].isnull() == True].index
for i in null_idx:
    print(df.loc[i, '충전시간'], df.loc[i, '충전시작일시(변환)'])

-6:-2:-5 2020-07-01 19:35:59
00:-4:-1 2020-07-05 15:39:18
00:-1:-4 2020-07-06 09:05:47
00:-1:-2 2020-07-06 09:05:47
00:-4:-2 2021-12-30 17:18:27
00:-1:-3 2022-01-14 17:53:48
-1:-4:-4 2022-08-19 03:00:11


In [141]:
df.shape

(11665822, 26)

In [142]:
df2 = df.drop(null_idx).reset_index(drop=True)
df2.shape

(11665815, 26)

In [143]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665815 entries, 0 to 11665814
Data columns (total 26 columns):
 #   Column      Dtype          
---  ------      -----          
 0   충전소명        object         
 1   충전기ID       int16          
 2   지역          object         
 3   시군구         object         
 4   주소          object         
 5   충전기타입       object         
 6   충전시작일시      int64          
 7   충전종료일시      object         
 8   충전시간        object         
 9   충전량         float64        
 10  충전용량        object         
 11  충전시작_년도     int32          
 12  충전시작_월      int32          
 13  충전시작_일      int32          
 14  충전시작_시      int32          
 15  충전시작_분      int32          
 16  충전시작_초      int32          
 17  충전시작일시(변환)  datetime64[ns] 
 18  충전종료_년도     int32          
 19  충전종료_월      int32          
 20  충전종료_일      int32          
 21  충전종료_시      int32          
 22  충전종료_분      int32          
 23  충전종료_초      int32          
 24  충전종료일시(변환)  datetime64

In [144]:
df2['충전시간'] = pd.to_timedelta(df2['충전시간'])
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665815 entries, 0 to 11665814
Data columns (total 26 columns):
 #   Column      Dtype          
---  ------      -----          
 0   충전소명        object         
 1   충전기ID       int16          
 2   지역          object         
 3   시군구         object         
 4   주소          object         
 5   충전기타입       object         
 6   충전시작일시      int64          
 7   충전종료일시      object         
 8   충전시간        timedelta64[ns]
 9   충전량         float64        
 10  충전용량        object         
 11  충전시작_년도     int32          
 12  충전시작_월      int32          
 13  충전시작_일      int32          
 14  충전시작_시      int32          
 15  충전시작_분      int32          
 16  충전시작_초      int32          
 17  충전시작일시(변환)  datetime64[ns] 
 18  충전종료_년도     int32          
 19  충전종료_월      int32          
 20  충전종료_일      int32          
 21  충전종료_시      int32          
 22  충전종료_분      int32          
 23  충전종료_초      int32          
 24  충전종료일시(변환)  datetime64

In [145]:
df2['충전시간'].describe()

count                     11665815
mean     0 days 00:34:21.381945453
std      0 days 01:20:05.913477577
min                0 days 00:00:00
25%                0 days 00:20:55
50%                0 days 00:34:07
75%                0 days 00:40:00
max                4 days 03:56:55
Name: 충전시간, dtype: object

In [146]:
df2['충전시간(계산)'].describe()

count                     11665815
mean     0 days 00:29:18.506520633
std      4 days 08:44:42.009406092
min          -14905 days +09:10:08
25%                0 days 00:21:28
50%                0 days 00:34:49
75%                0 days 00:40:24
max               14 days 08:40:25
Name: 충전시간(계산), dtype: object

In [147]:
df2[df2['충전시간(계산)'] == '-14905 days +09:10:08']

,충전소명,충전기ID,지역,시군구,주소,충전기타입,충전시작일시,충전종료일시,충전시간,충전량,...,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산)
2154135,단양관광안내소(단양팔경),1,충청북도,단양군,충청북도 단양군 단양읍 고수리 149-1,DC차데모+AC3상+DC콤보,20201030144953,19800110000001,3 days 10:47:44,14.20000,...,53,2020-10-30 14:49:53,1980,1,10,0,0,1,1980-01-10 00:00:01,-14905 days +09:10:08


##### 충전시간 오류 제거 데이터 출력

In [156]:
df2.to_csv('2020-22년_급속충전_충전시간_오류제거.csv', encoding='utf-8')

## 데이터 로드

In [14]:
df2 = pd.read_csv('2020-22년_급속충전_충전시간_오류제거.csv', index_col=0, dtype={'충전시작_년도':np.int16, '충전종료_년도':np.int16, '충전시작_월':np.int8,  '충전시작_일':np.int8, '충전시작_시':np.int8, '충전시작_분':np.int8, '충전시작_초':np.int8, '충전종료_월':np.int8, '충전종료_일':np.int8, '충전종료_시':np.int8, '충전종료_분':np.int8, '충전종료_초':np.int8, '충전기ID':np.int8})
df2['충전시간'] = pd.to_timedelta(df2['충전시간'])
df2['충전시간(계산)'] = pd.to_timedelta(df2['충전시간(계산)'])
df2 = df2.reset_index(drop=True)
df2.info()

C:\Users\kbj\AppData\Local\Temp\ipykernel_13392\2104362447.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('2020-22년_급속충전_충전시간_오류제거.csv', index_col=0, dtype={'충전시작_년도':np.int16, '충전종료_년도':np.int16, '충전시작_월':np.int8,  '충전시작_일':np.int8, '충전시작_시':np.int8, '충전시작_분':np.int8, '충전시작_초':np.int8, '충전종료_월':np.int8, '충전종료_일':np.int8, '충전종료_시':np.int8, '충전종료_분':np.int8, '충전종료_초':np.int8, '충전기ID':np.int8})


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665815 entries, 0 to 11665814
Data columns (total 26 columns):
 #   Column      Dtype          
---  ------      -----          
 0   충전소명        object         
 1   충전기ID       int8           
 2   지역          object         
 3   시군구         object         
 4   주소          object         
 5   충전기타입       object         
 6   충전시작일시      int64          
 7   충전종료일시      int64          
 8   충전시간        timedelta64[ns]
 9   충전량         float64        
 10  충전용량        object         
 11  충전시작_년도     int16          
 12  충전시작_월      int8           
 13  충전시작_일      int8           
 14  충전시작_시      int8           
 15  충전시작_분      int8           
 16  충전시작_초      int8           
 17  충전시작일시(변환)  object         
 18  충전종료_년도     int16          
 19  충전종료_월      int8           
 20  충전종료_일      int8           
 21  충전종료_시      int8           
 22  충전종료_분      int8           
 23  충전종료_초      int8           
 24  충전종료일시(변환)  object    

In [15]:
count = 0
for one in df2['충전소명']:
    if '"' in one:
        count += 1
print(count)

0


### 연도/지역별 충전기ID 개수
- 충전시간(계산) 음수 값 유지

In [57]:
numch_col = ['충전시작_년도', '충전소명', '충전기ID']
numch = df2.drop_duplicates(numch_col, keep='first').reset_index(drop=True)
numch.shape

(17585, 26)

In [80]:
numch_group_chargerID = numch.groupby(['충전시작_년도', '지역', '시군구', '충전기ID'])['충전소명'].count().rename('시군구별 충전기ID별 개수')
numch_group_chargerID

충전시작_년도  지역    시군구  충전기ID
2020     강원도   강릉시  1        15
                    2        11
                    3         5
                    4         1
                    21        1
                             ..
2022     충청북도  충주시  44        2
                    45        1
                    81        1
                    82        1
                    91        1
Name: 충전기ID 수, Length: 5040, dtype: int64

In [81]:
numch_group_chargerID_y = numch.groupby(['지역', '시군구', '충전기ID', '충전시작_년도'])['충전소명'].count().rename('충전기ID별 개수')
numch_group_chargerID_y

지역    시군구  충전기ID  충전시작_년도
강원도   강릉시  1      2020       15
                  2021       17
                  2022       20
           2      2020       11
                  2021       11
                             ..
충청북도  충주시  81     2022        1
           82     2022        1
           91     2020        1
                  2021        1
                  2022        1
Name: 충전기ID 수, Length: 5040, dtype: int64

In [82]:
numch_group_chargerID_y_u = numch.groupby(['지역', '시군구', '충전시작_년도'])['충전기ID'].unique().rename('충전기ID unique값')
numch_group_chargerID_y_u

지역    시군구  충전시작_년도
강원도   강릉시  2020                [4, 1, 3, 2, 52, 42, 41, 43, 44, 21, 22]
           2021            [42, 3, 1, 52, 2, 43, 41, 22, 4, 21, 44, 45]
           2022               [1, 2, 22, 3, 21, 42, 45, 43, 41, 52, 44]
      고성군  2020                                          [12, 11, 1, 2]
           2021                                          [12, 2, 1, 11]
                                            ...                        
충청북도  청주시  2021          [1, 43, 2, 12, 11, 42, 52, 41, 21, 22, 45, 44]
           2022       [1, 12, 41, 11, 2, 21, 22, 52, 44, 42, 45, 43,...
      충주시  2020          [1, 11, 12, 2, 21, 22, 91, 41, 42, 45, 44, 43]
           2021       [1, 12, 21, 22, 42, 41, 2, 43, 11, 44, 45, 91,...
           2022       [1, 12, 21, 22, 91, 2, 11, 44, 42, 45, 24, 43,...
Name: 충전기ID 값, Length: 744, dtype: object

In [83]:
numch_group_chargerID_y_u.shape

(744,)

In [84]:
numch_group_chargerID_y_u.iloc[0]

array([ 4,  1,  3,  2, 52, 42, 41, 43, 44, 21, 22], dtype=int8)

In [85]:
for i, one in enumerate(numch_group_chargerID_y_u):
    numch_group_chargerID_y_u.iloc[i] = sorted(one)

In [86]:
numch_group_chargerID_y_u

지역    시군구  충전시작_년도
강원도   강릉시  2020                [1, 2, 3, 4, 21, 22, 41, 42, 43, 44, 52]
           2021            [1, 2, 3, 4, 21, 22, 41, 42, 43, 44, 45, 52]
           2022               [1, 2, 3, 21, 22, 41, 42, 43, 44, 45, 52]
      고성군  2020                                          [1, 2, 11, 12]
           2021                                          [1, 2, 11, 12]
                                            ...                        
충청북도  청주시  2021          [1, 2, 11, 12, 21, 22, 41, 42, 43, 44, 45, 52]
           2022       [1, 2, 3, 11, 12, 21, 22, 23, 24, 41, 42, 43, ...
      충주시  2020          [1, 2, 11, 12, 21, 22, 41, 42, 43, 44, 45, 91]
           2021       [1, 2, 11, 12, 21, 22, 23, 24, 41, 42, 43, 44,...
           2022       [1, 2, 3, 4, 11, 12, 21, 22, 23, 24, 41, 42, 4...
Name: 충전기ID 값, Length: 744, dtype: object

In [87]:
numch_group_chargerID_y_nu = numch.groupby(['지역', '시군구', '충전시작_년도'])['충전기ID'].nunique().rename('충전기ID nunique')
numch_group_chargerID_y_nu

지역    시군구  충전시작_년도
강원도   강릉시  2020       11
           2021       12
           2022       11
      고성군  2020        4
           2021        4
                      ..
충청북도  청주시  2021       12
           2022       15
      충주시  2020       12
           2021       14
           2022       18
Name: 충전기ID 개수, Length: 744, dtype: int64

In [77]:
numch_group_chargerID_sum = numch.groupby(['충전시작_년도', '지역', '충전기ID'])['충전소명'].count().rename('지역별 ID별 충전기 개수')
numch_group_chargerID_sum

충전시작_년도  지역    충전기ID
2020     강원도   1        158
               2         56
               3         10
               4          2
               11        39
                       ... 
2022     충청북도  45         2
               52         8
               81         2
               82         2
               91         5
Name: 충전소명, Length: 1015, dtype: int64

In [88]:
numch_group_chargerID_file = pd.ExcelWriter('분석/2020-22년_년도_지역별_충전기ID_수2.xlsx')
numch_group_chargerID.to_excel(numch_group_chargerID_file, sheet_name='시군구별 충전기ID별 개수')
numch_group_chargerID_y.to_excel(numch_group_chargerID_file, sheet_name='충전기ID별 개수')
numch_group_chargerID_y_u.to_excel(numch_group_chargerID_file, sheet_name='충전기ID unique값')
numch_group_chargerID_y_nu.to_excel(numch_group_chargerID_file, sheet_name='충전기ID nunique')
numch_group_chargerID_sum.to_excel(numch_group_chargerID_file, sheet_name='지역별 ID별 충전기 개수')
numch_group_chargerID_file.save()

#### 충전시간 데이터 확인

In [148]:
df2[df2['충전시간'] == '0 days 00:00:00'].head()

,충전소명,충전기ID,지역,시군구,주소,충전기타입,충전시작일시,충전종료일시,충전시간,충전량,...,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산)
40755,스포츠파크 주차장,11,경상남도,거창군,경상남도 거창군 거창읍 심소정길39-36,DC콤보,20200106124403,20200106124403,0 days,1.02000,...,3,2020-01-06 12:44:03,2020,1,6,12,44,3,2020-01-06 12:44:03,0 days 00:00:00
48908,이마트 파주운정점,11,경기도,파주시,경기도 파주시 한울로 123,DC콤보,20200107165228,20200107165255,0 days,22.51000,...,28,2020-01-07 16:52:28,2020,1,7,16,52,55,2020-01-07 16:52:55,0 days 00:00:27
61941,(재)문화엑스포,11,경상북도,경주시,경상북도 경주시 경감로 614,DC콤보,20200109145420,20200109145420,0 days,1.18000,...,20,2020-01-09 14:54:20,2020,1,9,14,54,20,2020-01-09 14:54:20,0 days 00:00:00
103629,종합사회복지관,12,충청남도,공주시,충청남도 공주시 의당면 의당로 257,DC콤보,20200114192124,20200114192132,0 days,14.58000,...,24,2020-01-14 19:21:24,2020,1,14,19,21,32,2020-01-14 19:21:32,0 days 00:00:08
156226,부안국민체육센터,11,전라북도,부안군,전라북도 부안군 부안읍 석정로 162,DC콤보,20200121162513,20200121162513,0 days,1.48000,...,13,2020-01-21 16:25:13,2020,1,21,16,25,13,2020-01-21 16:25:13,0 days 00:00:00


In [149]:
df2[df2['충전시간'] == '0 days 00:00:00'].shape

(5702, 26)

In [150]:
df2['충전량'].dtype

dtype('float64')

In [151]:
df2['충전량'].head()

0   20.22000
1   18.87000
2   16.04000
3    5.80000
4   13.32000
Name: 충전량, dtype: float64

In [152]:
df2[ (df2['충전시간'] == '0 days 00:00:00') & (df2['충전량'] != 0) ][['충전시간', '충전량']]

,충전시간,충전량
40755,0 days,1.02000
48908,0 days,22.51000
61941,0 days,1.18000
103629,0 days,14.58000
156226,0 days,1.48000
...,...,...
11571981,0 days,18.70000
11572687,0 days,23.41000
11607861,0 days,7.44000
11618040,0 days,4.50000


In [153]:
df2.loc[ (df2['충전시간'] == '0 days 00:00:00') & (df2['충전량'] != 0), ['충전시간', '충전량'] ]

,충전시간,충전량
40755,0 days,1.02000
48908,0 days,22.51000
61941,0 days,1.18000
103629,0 days,14.58000
156226,0 days,1.48000
...,...,...
11571981,0 days,18.70000
11572687,0 days,23.41000
11607861,0 days,7.44000
11618040,0 days,4.50000


In [154]:
df2[ (df2['충전시간'] == '0 days 00:00:00') & (df2['충전량'] != 0) ].shape

(5702, 26)

#### 충전시간(계산) 음수 값 제거
- 52366개

In [155]:
n_ch_time_idx = df2[df2['충전시간(계산)'] < '0'].index
len(n_ch_time_idx)

52366

In [158]:
df2[df2['충전시간(계산)'] < '0']['충전시간(계산)'].head()

16367   -1 days +14:47:54
16370   -1 days +14:47:04
16373   -1 days +14:46:20
16935   -1 days +13:26:06
16941   -1 days +13:25:25
Name: 충전시간(계산), dtype: timedelta64[ns]

In [159]:
df2[df2['충전시간(계산)'] < '0']['충전시간(계산)'].tail()

3255838    -1 days +23:54:45
7653730    -1 days +23:55:24
8325194    -1 days +22:25:53
9172214    -1 days +23:38:16
11279091   -1 days +21:45:24
Name: 충전시간(계산), dtype: timedelta64[ns]

In [157]:
df2.shape

(11665815, 26)

In [160]:
df2 = df2.drop(n_ch_time_idx).reset_index(drop=True)

In [161]:
df2.shape

(11613449, 26)

In [162]:
df2['충전시간'].describe()

count                     11613449
mean     0 days 00:33:51.200344617
std      0 days 01:19:04.781739278
min                0 days 00:00:00
25%                0 days 00:20:53
50%                0 days 00:34:02
75%                0 days 00:40:00
max                4 days 03:56:55
Name: 충전시간, dtype: object

In [163]:
df2['충전시간(계산)'].describe()

count                     11613449
mean     0 days 00:34:58.600275335
std      0 days 01:23:19.668917229
min                0 days 00:00:00
25%                0 days 00:21:39
50%                0 days 00:34:56
75%                0 days 00:40:25
max               14 days 08:40:25
Name: 충전시간(계산), dtype: object

#### 원본 충전시간과 계산 충전시간 차이 통계

In [164]:
temp = df2['충전시간(계산)'] - df2['충전시간']
temp.describe()

count                     11613449
mean     0 days 00:01:07.399930718
std      0 days 01:01:06.908441070
min              -1 days +00:00:23
25%                0 days 00:00:00
50%                0 days 00:00:13
75%                0 days 00:01:42
max               14 days 08:32:12
dtype: object

In [165]:
temp[temp < '0'].shape

(817340,)

In [166]:
df2['raw값과_계산값_차이'] = df2['충전시간(계산)'] - df2['충전시간']

In [167]:
df2[df2['raw값과_계산값_차이'] > '14 days']

,충전소명,충전기ID,지역,시군구,주소,충전기타입,충전시작일시,충전종료일시,충전시간,충전량,...,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산),raw값과_계산값_차이
8083907,포항실내사격장,11,경상북도,포항시,경상북도 포항시 북구 용흥동 4-9,DC콤보,20220305094551,20220319182616,0 days 00:08:13,3.24000,...,2022-03-05 09:45:51,2022,3,19,18,26,16,2022-03-19 18:26:16,14 days 08:40:25,14 days 08:32:12


In [168]:
# df2 = df2.drop(['raw값과_계산값_차이'], axis=1)

In [169]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11613449 entries, 0 to 11613448
Data columns (total 27 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int16          
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기타입         object         
 6   충전시작일시        int64          
 7   충전종료일시        object         
 8   충전시간          timedelta64[ns]
 9   충전량           float64        
 10  충전용량          object         
 11  충전시작_년도       int32          
 12  충전시작_월        int32          
 13  충전시작_일        int32          
 14  충전시작_시        int32          
 15  충전시작_분        int32          
 16  충전시작_초        int32          
 17  충전시작일시(변환)    datetime64[ns] 
 18  충전종료_년도       int32          
 19  충전종료_월        int32          
 20  충전종료_일        int32          
 21  충전종료_시        int32          
 22  충전종료_분        int32          
 23  충전종료_

In [170]:
df2.iloc[:, :14].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11613449 entries, 0 to 11613448
Data columns (total 14 columns):
 #   Column   Dtype          
---  ------   -----          
 0   충전소명     object         
 1   충전기ID    int16          
 2   지역       object         
 3   시군구      object         
 4   주소       object         
 5   충전기타입    object         
 6   충전시작일시   int64          
 7   충전종료일시   object         
 8   충전시간     timedelta64[ns]
 9   충전량      float64        
 10  충전용량     object         
 11  충전시작_년도  int32          
 12  충전시작_월   int32          
 13  충전시작_일   int32          
dtypes: float64(1), int16(1), int32(3), int64(1), object(7), timedelta64[ns](1)
memory usage: 1.0+ GB


In [171]:
df2.iloc[:, 14:].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11613449 entries, 0 to 11613448
Data columns (total 13 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전시작_시        int32          
 1   충전시작_분        int32          
 2   충전시작_초        int32          
 3   충전시작일시(변환)    datetime64[ns] 
 4   충전종료_년도       int32          
 5   충전종료_월        int32          
 6   충전종료_일        int32          
 7   충전종료_시        int32          
 8   충전종료_분        int32          
 9   충전종료_초        int32          
 10  충전종료일시(변환)    datetime64[ns] 
 11  충전시간(계산)      timedelta64[ns]
 12  raw값과_계산값_차이  timedelta64[ns]
dtypes: datetime64[ns](2), int32(9), timedelta64[ns](2)
memory usage: 753.1 MB


### 데이터 절약

In [172]:
time_cols = ['충전시작_년도', '충전시작_월',  '충전시작_일', '충전시작_시', '충전시작_분', '충전시작_초', '충전종료_년도', '충전종료_월', '충전종료_일', '충전종료_시', '충전종료_분', '충전종료_초']
df2[time_cols].describe()

,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초
count,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000,11613449.00000
mean,2021.16059,6.20425,15.80842,13.72640,29.47740,29.49963,2021.16062,6.20430,15.80841,14.01221,29.54277,29.45744
std,0.77937,3.33015,8.80106,4.95829,17.33414,17.31996,0.77937,3.33017,8.80110,5.07576,17.34527,17.33907
min,2020.00000,1.00000,1.00000,0.00000,0.00000,0.00000,2020.00000,1.00000,1.00000,0.00000,0.00000,0.00000
25%,2021.00000,3.00000,8.00000,10.00000,14.00000,14.00000,2021.00000,3.00000,8.00000,11.00000,15.00000,14.00000
50%,2021.00000,6.00000,16.00000,14.00000,29.00000,29.00000,2021.00000,6.00000,16.00000,14.00000,30.00000,29.00000
75%,2022.00000,9.00000,23.00000,17.00000,45.00000,45.00000,2022.00000,9.00000,23.00000,18.00000,45.00000,44.00000
max,2022.00000,12.00000,31.00000,23.00000,59.00000,59.00000,2022.00000,12.00000,31.00000,23.00000,59.00000,59.00000


In [173]:
df2[['충전시작_년도', '충전종료_년도']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11613449 entries, 0 to 11613448
Data columns (total 2 columns):
 #   Column   Dtype
---  ------   -----
 0   충전시작_년도  int32
 1   충전종료_년도  int32
dtypes: int32(2)
memory usage: 88.6 MB


In [174]:
df2[['충전시작_년도', '충전종료_년도']] = df2[['충전시작_년도', '충전종료_년도']].astype('int16')

In [175]:
df2[['충전시작_년도', '충전종료_년도']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11613449 entries, 0 to 11613448
Data columns (total 2 columns):
 #   Column   Dtype
---  ------   -----
 0   충전시작_년도  int16
 1   충전종료_년도  int16
dtypes: int16(2)
memory usage: 44.3 MB


In [55]:
charge_cols = ['충전시작_월',  '충전시작_일', '충전시작_시', '충전시작_분', '충전시작_초', '충전종료_월', '충전종료_일', '충전종료_시', '충전종료_분', '충전종료_초']
df2[charge_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11665821
Data columns (total 10 columns):
 #   Column  Dtype
---  ------  -----
 0   충전시작_월  int64
 1   충전시작_일  int64
 2   충전시작_시  int64
 3   충전시작_분  int64
 4   충전시작_초  int64
 5   충전종료_월  int64
 6   충전종료_일  int64
 7   충전종료_시  int64
 8   충전종료_분  int64
 9   충전종료_초  int64
dtypes: int64(10)
memory usage: 974.6 MB


In [176]:
df2[charge_cols] = df2[charge_cols].astype('int8')
df2[charge_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11613449 entries, 0 to 11613448
Data columns (total 10 columns):
 #   Column  Dtype
---  ------  -----
 0   충전시작_월  int8 
 1   충전시작_일  int8 
 2   충전시작_시  int8 
 3   충전시작_분  int8 
 4   충전시작_초  int8 
 5   충전종료_월  int8 
 6   충전종료_일  int8 
 7   충전종료_시  int8 
 8   충전종료_분  int8 
 9   충전종료_초  int8 
dtypes: int8(10)
memory usage: 110.8 MB


In [177]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11613449 entries, 0 to 11613448
Data columns (total 27 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int16          
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기타입         object         
 6   충전시작일시        int64          
 7   충전종료일시        object         
 8   충전시간          timedelta64[ns]
 9   충전량           float64        
 10  충전용량          object         
 11  충전시작_년도       int16          
 12  충전시작_월        int8           
 13  충전시작_일        int8           
 14  충전시작_시        int8           
 15  충전시작_분        int8           
 16  충전시작_초        int8           
 17  충전시작일시(변환)    datetime64[ns] 
 18  충전종료_년도       int16          
 19  충전종료_월        int8           
 20  충전종료_일        int8           
 21  충전종료_시        int8           
 22  충전종료_분        int8           
 23  충전종료_

In [178]:
df2['충전기ID'].describe()

count   11613449.00000
mean           8.59347
std           15.08407
min            1.00000
25%            1.00000
50%            1.00000
75%           12.00000
max           98.00000
Name: 충전기ID, dtype: float64

In [179]:
df2['충전기ID'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 11613449 entries, 0 to 11613448
Series name: 충전기ID
Non-Null Count     Dtype
--------------     -----
11613449 non-null  int16
dtypes: int16(1)
memory usage: 22.2 MB


In [64]:
df2['충전시간'].describe()

count                     11613449
mean     0 days 00:33:51.200344617
std      0 days 01:19:04.781739278
min                0 days 00:00:00
25%                0 days 00:20:53
50%                0 days 00:34:02
75%                0 days 00:40:00
max                4 days 03:56:55
Name: 충전시간, dtype: object

## 충전시간 정리 파일로 출력

In [181]:
df2.to_csv('2020-22년_급속충전-충전시간정리.csv', encoding='utf-8')

In [182]:
# df2.to_excel('2020-22년_급속충전-충전시간정리.xlsx', encoding='utf-8') # error : 파일 너무 커서 저장 실패

## 충전시간 정리 파일 로드(다시 시작)

In [4]:
# df2 = pd.read_csv('2020-22년_급속충전-충전시간정리.csv', index_col=0, dtype={'충전시작_년도':np.int16, '충전종료_년도':np.int16, '충전시작_월':np.int8,  '충전시작_일':np.int8, '충전시작_시':np.int8, '충전시작_분':np.int8, '충전시작_초':np.int8, '충전종료_월':np.int8, '충전종료_일':np.int8, '충전종료_시':np.int8, '충전종료_분':np.int8, '충전종료_초':np.int8, '충전기ID':np.int8})
# df2['충전시간'] = pd.to_timedelta(df2['충전시간'])
# df2['충전시간(계산)'] = pd.to_timedelta(df2['충전시간(계산)'])
# df2 = df2.reset_index(drop=True)
# df2.info()

C:\Users\kbj\AppData\Local\Temp\ipykernel_7108\3489641763.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('2020-22년_급속충전-충전시간정리.csv', index_col=0, dtype={'충전시작_년도':np.int16, '충전종료_년도':np.int16, '충전시작_월':np.int8,  '충전시작_일':np.int8, '충전시작_시':np.int8, '충전시작_분':np.int8, '충전시작_초':np.int8, '충전종료_월':np.int8, '충전종료_일':np.int8, '충전종료_시':np.int8, '충전종료_분':np.int8, '충전종료_초':np.int8, '충전기ID':np.int8})


<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613449 entries, 0 to 11613448
Data columns (total 27 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int8           
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기타입         object         
 6   충전시작일시        int64          
 7   충전종료일시        int64          
 8   충전시간          timedelta64[ns]
 9   충전량           float64        
 10  충전용량          object         
 11  충전시작_년도       int16          
 12  충전시작_월        int8           
 13  충전시작_일        int8           
 14  충전시작_시        int8           
 15  충전시작_분        int8           
 16  충전시작_초        int8           
 17  충전시작일시(변환)    object         
 18  충전종료_년도       int16          
 19  충전종료_월        int8           
 20  충전종료_일        int8           
 21  충전종료_시        int8           
 22  충전종료_분        int8           
 23  충전종료_

#### 데이터 절약

In [5]:
# df2[['충전시작_년도', '충전종료_년도']] = df2[['충전시작_년도', '충전종료_년도']].astype('int16')
# charge_cols = ['충전시작_월',  '충전시작_일', '충전시작_시', '충전시작_분', '충전시작_초', '충전종료_월', '충전종료_일', '충전종료_시', '충전종료_분', '충전종료_초']
# df2[charge_cols] = df2[charge_cols].astype('int8')
# df2['충전기ID'] = df2['충전기ID'].astype('int8')
# df2[charge_cols].info()

## 분석

### 년도/지역별 충전시간, 충전량 통계
- 총 충전시간, 총 충전량

#### 연도, 지역별 총 충전시간

In [9]:
total_charge_time_l = df2.groupby(['지역', '시군구'])['충전시간'].sum()
total_charge_time_l

지역    시군구
강원도   강릉시   2000 days 10:06:29
      고성군    342 days 10:02:43
      동해시   1281 days 14:11:21
      삼척시    801 days 16:11:31
      속초시    894 days 04:56:14
                   ...        
충청북도  제천시   1138 days 18:12:40
      증평군    154 days 17:00:42
      진천군   1108 days 10:33:16
      청주시   5926 days 17:03:39
      충주시   3080 days 13:36:25
Name: 충전시간, Length: 252, dtype: timedelta64[ns]

In [6]:
total_charge_time_y = df2.groupby(['충전시작_년도'])['충전시간'].sum()
total_charge_time_y

충전시작_년도
2020             89296 days 01:56:46
2021             88766 days 15:01:20
2022   94960 days 22:08:44.999999488
Name: 충전시간, dtype: timedelta64[ns]

In [7]:
total_charge_time_yl = df2.groupby(['충전시작_년도', '지역'])['충전시간'].sum()
total_charge_time_yl

충전시작_년도  지역     
2020     강원도        6148 days 10:01:35
         경기도       13840 days 18:55:17
         경상남도       4730 days 02:41:21
         경상북도      11275 days 13:47:49
         광주광역시      1824 days 08:01:38
         대구광역시      3112 days 19:53:59
         대전광역시      2662 days 07:41:09
         부산광역시      2020 days 11:30:21
         서울특별시      6382 days 09:09:38
         세종특별자치시     568 days 20:38:10
         울산광역시      1634 days 16:25:51
         인천광역시      3482 days 21:46:27
         전라남도       5581 days 05:29:41
         전라북도       5187 days 09:00:55
         제주특별자치도    8284 days 00:42:42
         충청남도       6550 days 05:43:33
         충청북도       6009 days 12:26:40
2021     강원도        6241 days 13:48:26
         경기도       14241 days 04:13:44
         경상남도       7025 days 11:04:01
         경상북도      10720 days 02:23:57
         광주광역시      1537 days 21:59:15
         대구광역시      3023 days 14:48:01
         대전광역시      2665 days 16:03:49
         부산광역시      2225 days 08:29:06
        

In [8]:
total_charge_time_yl2 = df2.groupby(['충전시작_년도', '지역', '시군구'])['충전시간'].sum()
total_charge_time_yl2

충전시작_년도  지역    시군구
2020     강원도   강릉시    410 days 15:28:29
               고성군     62 days 20:02:16
               동해시    708 days 22:49:15
               삼척시    210 days 20:42:17
               속초시    194 days 05:11:37
                            ...        
2022     충청북도  제천시    393 days 12:09:25
               증평군     62 days 12:48:55
               진천군    271 days 04:46:10
               청주시   2235 days 01:32:44
               충주시   1141 days 00:21:33
Name: 충전시간, Length: 744, dtype: timedelta64[ns]

##### 연도/지역별 총 충전시간 파일 출력

In [10]:
total_charge_time_group = pd.ExcelWriter('분석/2020-22년_연도_지역별_충전시간.xlsx')
total_charge_time_l.to_excel(total_charge_time_group, sheet_name='시군구별')
total_charge_time_y.to_excel(total_charge_time_group, sheet_name='년도별')
total_charge_time_yl.to_excel(total_charge_time_group, sheet_name='년도지역별')
total_charge_time_yl2.to_excel(total_charge_time_group, sheet_name='년도시군구별')
total_charge_time_group.save()

#### 지역/연도별 총 충전량 합계

In [11]:
total_charge_vol_l = df2.groupby(['지역', '시군구'])['충전량'].sum()
total_charge_vol_l

지역    시군구
강원도   강릉시   1857604.33000
      고성군    369691.09000
      동해시    701535.12000
      삼척시    716323.83000
      속초시    860865.62000
                 ...     
충청북도  제천시    999654.99000
      증평군    155696.15000
      진천군    591631.10000
      청주시   5178912.01000
      충주시   2493316.51000
Name: 충전량, Length: 252, dtype: float64

In [12]:
total_charge_vol_y = df2.groupby(['충전시작_년도'])['충전량'].sum()
total_charge_vol_y

충전시작_년도
2020   46811133.65000
2021   80999447.99000
2022   97428562.80000
Name: 충전량, dtype: float64

In [13]:
total_charge_vol_yl = df2.groupby(['충전시작_년도', '지역'])['충전량'].sum()
total_charge_vol_yl

충전시작_년도  지역     
2020     강원도        2989184.45000
         경기도        7010474.66000
         경상남도       3167694.35000
         경상북도       5910919.27000
         광주광역시       754169.55000
         대구광역시      2146057.41000
         대전광역시      1293125.47000
         부산광역시      1062800.58000
         서울특별시      3316642.81000
         세종특별자치시     317304.54000
         울산광역시       780398.72000
         인천광역시      1348635.13000
         전라남도       2539533.61000
         전라북도       2497671.17000
         제주특별자치도    4802271.95000
         충청남도       3916231.72000
         충청북도       2958018.26000
2021     강원도        5871167.90000
         경기도       13035269.15000
         경상남도       6517862.71000
         경상북도      10042405.83000
         광주광역시      1445372.51000
         대구광역시      2783647.75000
         대전광역시      2351933.04000
         부산광역시      1921385.39000
         서울특별시      5851317.95000
         세종특별자치시     525703.96000
         울산광역시      1304336.52000
         인천광역시      3110031.030

In [14]:
total_charge_vol_yl2 = df2.groupby(['충전시작_년도', '지역', '시군구'])['충전량'].sum()
total_charge_vol_yl2

충전시작_년도  지역    시군구
2020     강원도   강릉시    298971.29000
               고성군     47163.01000
               동해시     91053.80000
               삼척시    130089.39000
               속초시    142168.72000
                          ...     
2022     충청북도  제천시    416637.20000
               증평군     63544.97000
               진천군    277948.46000
               청주시   2263350.46000
               충주시   1161811.42000
Name: 충전량, Length: 744, dtype: float64

##### 년도/지역별 총 충전량 출력

In [15]:
total_charge_vol_group = pd.ExcelWriter('분석/2020-22년_연도_지역별_충전량.xlsx')
total_charge_vol_l.to_excel(total_charge_vol_group, sheet_name='시군구별')
total_charge_vol_y.to_excel(total_charge_vol_group, sheet_name='년도별')
total_charge_vol_yl.to_excel(total_charge_vol_group, sheet_name='년도지역별')
total_charge_vol_yl2.to_excel(total_charge_vol_group, sheet_name='년도시군구별')
total_charge_vol_group.save()

### 년도, 월별 충전시간, 충전량 통계
- 총 충전시간, 총 충전량

#### 년도/월별 총 충전시간

In [17]:
charge_time_y = df2.groupby(['충전시작_년도'])['충전시간'].sum()
charge_time_y

충전시작_년도
2020             89296 days 01:56:46
2021             88766 days 15:01:20
2022   94960 days 22:08:44.999999488
Name: 충전시간, dtype: timedelta64[ns]

In [16]:
charge_time_ym = df2.groupby(['충전시작_년도', '충전시작_월'])['충전시간'].sum()
charge_time_ym

충전시작_년도  충전시작_월
2020     1         4844 days 06:04:49
         2         3807 days 14:53:40
         3         4220 days 18:42:13
         4         5200 days 19:02:23
         5         6698 days 16:05:39
         6         8425 days 07:42:45
         7         9948 days 17:47:42
         8         9994 days 13:12:00
         9         7887 days 09:02:08
         10        9809 days 02:56:50
         11        9686 days 05:46:34
         12        8772 days 14:40:03
2021     1         6700 days 00:31:35
         2         5972 days 01:10:01
         3         6318 days 03:25:10
         4         5981 days 18:18:29
         5         5934 days 01:30:44
         6         5927 days 10:26:42
         7         6999 days 08:35:22
         8         7217 days 06:09:09
         9         7222 days 08:48:45
         10        8611 days 17:43:58
         11       10017 days 21:16:40
         12       11864 days 13:04:45
2022     1        11766 days 19:03:00
         2         9909 days 20:56

In [20]:
total_charge_time_group_ym = pd.ExcelWriter('분석/2020-22년_급속충전_년월별_총_충전시간.xlsx')
charge_time_y.to_excel(total_charge_time_group_ym, sheet_name='연도별')
charge_time_ym.to_excel(total_charge_time_group_ym, sheet_name='연도월별')
total_charge_time_group_ym.save()

#### 년도/월별 총 충전시간(계산)

In [21]:
cal_charge_time_y = df2.groupby(['충전시작_년도'])['충전시간(계산)'].sum()
cal_charge_time_y

충전시작_년도
2020    84050 days 00:29:12.999999488
2021    95417 days 01:17:47.000000512
2022   102616 days 02:40:48.999999488
Name: 충전시간(계산), dtype: timedelta64[ns]

In [22]:
cal_charge_time_ym = df2.groupby(['충전시작_년도', '충전시작_월'])['충전시간(계산)'].sum()
cal_charge_time_ym

충전시작_년도  충전시작_월
2020     1         5111 days 07:36:54
         2         4030 days 10:04:17
         3         4128 days 11:36:27
         4         4900 days 20:11:06
         5         6183 days 22:26:24
         6         7541 days 18:02:20
         7         8763 days 12:15:12
         8         8909 days 04:45:22
         9         7116 days 07:09:15
         10        9042 days 04:42:48
         11        9503 days 08:37:36
         12        8818 days 17:01:32
2021     1         6970 days 17:06:06
         2         6278 days 23:23:36
         3         6597 days 16:40:25
         4         6267 days 11:25:41
         5         6229 days 08:51:42
         6         6233 days 16:48:38
         7         7350 days 13:25:41
         8         7941 days 11:25:24
         9         8056 days 00:14:44
         10        9592 days 22:42:28
         11       10994 days 09:48:00
         12       12903 days 17:25:22
2022     1        12766 days 06:53:00
         2        10760 days 18:15

In [23]:
total_cal_charge_time_group_ym = pd.ExcelWriter('분석/2020-22년_급속충전_년월별_총_충전시간(계산).xlsx')
cal_charge_time_y.to_excel(total_cal_charge_time_group_ym, sheet_name='연도별')
cal_charge_time_ym.to_excel(total_cal_charge_time_group_ym, sheet_name='연도월별')
total_cal_charge_time_group_ym.save()

#### 년도/월별 총 충전량

In [24]:
cal_charge_vol_y = df2.groupby(['충전시작_년도'])['충전량'].sum()
cal_charge_vol_y

충전시작_년도
2020   46811133.65000
2021   80999447.99000
2022   97428562.80000
Name: 충전량, dtype: float64

In [25]:
cal_charge_vol_ym = df2.groupby(['충전시작_년도', '충전시작_월'])['충전량'].sum()
cal_charge_vol_ym

충전시작_년도  충전시작_월
2020     1         3728407.76000
         2         2923554.69000
         3         2868303.28000
         4         3029906.70000
         5         3364357.94000
         6         3562807.86000
         7         3794918.35000
         8         4398191.42000
         9         3893530.88000
         10        4855642.05000
         11        5046230.77000
         12        5345281.95000
2021     1         4985581.43000
         2         4688077.46000
         3         5284216.30000
         4         5254442.52000
         5         5396621.91000
         6         5646845.23000
         7         6912348.98000
         8         7154868.40000
         9         7228810.76000
         10        8412678.23000
         11        9455615.51000
         12       10579341.26000
2022     1        10302203.67000
         2         8714278.79000
         3        10292070.61000
         4        11457488.60000
         5        12850982.38000
         6        13187238.

In [26]:
total_charge_vol_group_ym = pd.ExcelWriter('분석/2020-22년_급속충전_년월별_총_충전량.xlsx')
cal_charge_vol_y.to_excel(total_charge_vol_group_ym, sheet_name='년도별')
cal_charge_vol_ym.to_excel(total_charge_vol_group_ym, sheet_name='년도월별')
total_charge_vol_group_ym.save()

## 데이터 로드(다시 시작)

In [16]:
df2 = pd.read_csv('2020-22년_급속충전-충전시간정리.csv', index_col=0, dtype={'충전시작_년도':np.int16, '충전종료_년도':np.int16, '충전시작_월':np.int8,  '충전시작_일':np.int8, '충전시작_시':np.int8, '충전시작_분':np.int8, '충전시작_초':np.int8, '충전종료_월':np.int8, '충전종료_일':np.int8, '충전종료_시':np.int8, '충전종료_분':np.int8, '충전종료_초':np.int8, '충전기ID':np.int8})
df2['충전시간'] = pd.to_timedelta(df2['충전시간'])
df2['충전시간(계산)'] = pd.to_timedelta(df2['충전시간(계산)'])
df2 = df2.reset_index(drop=True)
df2.info()

C:\Users\kbj\AppData\Local\Temp\ipykernel_13392\3053455079.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('2020-22년_급속충전-충전시간정리.csv', index_col=0, dtype={'충전시작_년도':np.int16, '충전종료_년도':np.int16, '충전시작_월':np.int8,  '충전시작_일':np.int8, '충전시작_시':np.int8, '충전시작_분':np.int8, '충전시작_초':np.int8, '충전종료_월':np.int8, '충전종료_일':np.int8, '충전종료_시':np.int8, '충전종료_분':np.int8, '충전종료_초':np.int8, '충전기ID':np.int8})


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11613449 entries, 0 to 11613448
Data columns (total 27 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int8           
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기타입         object         
 6   충전시작일시        int64          
 7   충전종료일시        int64          
 8   충전시간          timedelta64[ns]
 9   충전량           float64        
 10  충전용량          object         
 11  충전시작_년도       int16          
 12  충전시작_월        int8           
 13  충전시작_일        int8           
 14  충전시작_시        int8           
 15  충전시작_분        int8           
 16  충전시작_초        int8           
 17  충전시작일시(변환)    object         
 18  충전종료_년도       int16          
 19  충전종료_월        int8           
 20  충전종료_일        int8           
 21  충전종료_시        int8           
 22  충전종료_분        int8           
 23  충전종료_

## 주소 분리
- 샘플수
    - 신주소 : 7,755,902 개
    - 구주소 : 3,857,547 개
- 한 주소에 2개 다른 주소가 입력
    - 동일 지역
        - 신주소 : 20,837개
        - 구주소 : 99,756개
    - 다른 지역
        - 신주소 : 0개
        - 구주소 : 5,427개
    - raw데이터의 입력값 지역이 있는 주소로 병합
        - 예시) 전라남도 무안군 청계면 영산로 1854-16 전라남도 무안군 청계면 영산로 1854-16 -> 전라남도 무안군 청계면 영산로 1854-16
        - 예시) 충청남도 청양군 청양읍 군량리 533 충청남도 태안군 소원면 신덕리 272 -> 충청남도 청양군 청양읍 군량리 533
        - 예시) 전라북도 순창군 풍산면 반월리 12-4 충청남도 공주시 탄천면 삼각리 523-2 -> 전라북도 순창군 풍산면 반월리 12-4
- 도로명 수정
    - "도로명주소 안내" 참고하여 지역 코드와 맞추기위해
        - 예시) 첨단 벤처로 108번길 -> 첨단벤처로108번길

### 특이주소 처리
- '경상북도 경주시 천원2길 9 경북 경주시 교동 153-5'
    - 6463 개
- ~~지역과 주소가 다른 경우~~
    - ~~지역 : 제주특별자치도 제주시~~
    - ~~주소 : 강남구 양재대로31길~~
    
    => for문안에 정규표현식 적용시 변수명 잘못 지정하여 발생한 문제
- '충청북도 단양군 영춘면 온달평강3로 9 온달평강3로 9-1'

In [17]:
df2[(df2['지역'] != '서울특별시') & (df2['주소'] == '강남구 양재대로31길')]

,충전소명,충전기ID,지역,시군구,주소,충전기타입,충전시작일시,충전종료일시,충전시간,충전량,...,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산),raw값과_계산값_차이


In [26]:
sp_text = '경상북도 경주시 천원2길 9 경북 경주시 교동 153-5'
sp_idx = df2[df2['주소'] == sp_text].index
len(sp_idx)

0

In [19]:
sp_idx[:5]

Int64Index([5929, 6279, 6615, 7493, 9248], dtype='int64')

In [20]:
df2.loc[[5929, 6279]]['주소']

5929    경상북도 경주시 천원2길 9 경북 경주시 교동 153-5
6279    경상북도 경주시 천원2길 9 경북 경주시 교동 153-5
Name: 주소, dtype: object

In [21]:
sp_temp = df2.loc[sp_idx]
sp_temp.groupby(['충전시작_년도', '충전시작_월'])['주소'].count()

충전시작_년도  충전시작_월
2020     1         117
         2          93
         3          82
         4          73
         5          97
         6          94
         7         117
         8         175
         9          86
         10        112
         11        112
         12        133
2021     1         143
         2         142
         3         137
         4         138
         5         180
         6         196
         7         234
         8         252
         9         264
         10        275
         11        346
         12        262
2022     1         214
         2         262
         3         205
         4         388
         5         348
         6         303
         7         385
         8         498
Name: 주소, dtype: int64

In [22]:
for i in tqdm(sp_idx):
    if df2.loc[i, '주소'] == sp_text:
        df2.loc[i, '주소'] = '경상북도 경주시 천원2길 9'

100%|██████████| 6463/6463 [00:00<00:00, 10606.17it/s]


In [23]:
df2.loc[sp_idx]['주소'].head()

5929    경상북도 경주시 천원2길 9
6279    경상북도 경주시 천원2길 9
6615    경상북도 경주시 천원2길 9
7493    경상북도 경주시 천원2길 9
9248    경상북도 경주시 천원2길 9
Name: 주소, dtype: object

In [24]:
check_sp_idx = df2[df2['주소'] == sp_text].index
len(check_sp_idx)

0

In [27]:
sp_text2 = '충청북도 단양군 영춘면 온달평강3로 9 온달평강3로 9-1'
sp_idx2 = df2[df2['주소'] == sp_text2].index
len(sp_idx2)

808

In [28]:
sp_idx2[:5]

Int64Index([917059, 917123, 922961, 929169, 939062], dtype='int64')

In [29]:
df2.loc[[917059, 917123]]['주소']

917059    충청북도 단양군 영춘면 온달평강3로 9 온달평강3로 9-1
917123    충청북도 단양군 영춘면 온달평강3로 9 온달평강3로 9-1
Name: 주소, dtype: object

In [30]:
sp_temp2 = df2.loc[sp_idx2]
sp_temp2.groupby(['충전시작_년도', '충전시작_월'])['주소'].count()

충전시작_년도  충전시작_월
2020     5          5
         6         12
         7         12
         8         30
         9         25
         10        33
         11        27
         12        33
2021     1         50
         2         22
         3         26
         4         27
         5         26
         6         33
         7         53
         8         42
         9         31
         10        32
         11        27
         12        25
2022     1         18
         2         12
         3         10
         4         33
         5         49
         6         39
         7         38
         8         38
Name: 주소, dtype: int64

In [31]:
for i in tqdm(sp_idx2):
    if df2.loc[i, '주소'] == sp_text2:
        df2.loc[i, '주소'] = '충청북도 단양군 영춘면 온달평강3로 9'

100%|██████████| 808/808 [00:00<00:00, 8770.36it/s]


In [32]:
df2.loc[sp_idx2]['주소'].head()

917059    충청북도 단양군 영춘면 온달평강3로 9
917123    충청북도 단양군 영춘면 온달평강3로 9
922961    충청북도 단양군 영춘면 온달평강3로 9
929169    충청북도 단양군 영춘면 온달평강3로 9
939062    충청북도 단양군 영춘면 온달평강3로 9
Name: 주소, dtype: object

In [33]:
check_sp_idx = df2[df2['주소'] == sp_text2].index
len(check_sp_idx)

0

##### 이상 주소 처리 파일 출력

In [34]:
df2.to_csv('2020-22년_급속충전-충전시간정리2.csv', encoding='utf-8')

## 데이터 로드(이상 주소 처리)

In [35]:
# df2 = pd.read_csv('2020-22년_급속충전-충전시간정리2.csv', index_col=0, dtype={'충전시작_년도':np.int16, '충전종료_년도':np.int16, '충전시작_월':np.int8,  '충전시작_일':np.int8, '충전시작_시':np.int8, '충전시작_분':np.int8, '충전시작_초':np.int8, '충전종료_월':np.int8, '충전종료_일':np.int8, '충전종료_시':np.int8, '충전종료_분':np.int8, '충전종료_초':np.int8, '충전기ID':np.int8})
# df2['충전시간'] = pd.to_timedelta(df2['충전시간'])
# df2['충전시간(계산)'] = pd.to_timedelta(df2['충전시간(계산)'])
# df2 = df2.reset_index(drop=True)
# df2.info()

### 신/구주소 분리

In [36]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11613449 entries, 0 to 11613448
Data columns (total 27 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int8           
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기타입         object         
 6   충전시작일시        int64          
 7   충전종료일시        int64          
 8   충전시간          timedelta64[ns]
 9   충전량           float64        
 10  충전용량          object         
 11  충전시작_년도       int16          
 12  충전시작_월        int8           
 13  충전시작_일        int8           
 14  충전시작_시        int8           
 15  충전시작_분        int8           
 16  충전시작_초        int8           
 17  충전시작일시(변환)    object         
 18  충전종료_년도       int16          
 19  충전종료_월        int8           
 20  충전종료_일        int8           
 21  충전종료_시        int8           
 22  충전종료_분        int8           
 23  충전종료_

In [37]:
id(df2['주소'])

2312340112912

In [38]:
ad = df2['주소'].copy()

In [39]:
id(ad), id(df2), id(df2['주소'])

(2307466824624, 2300710790864, 2312340112912)

In [40]:
temp = ad.to_numpy()
len(temp)

11613449

In [41]:
old_p = re.compile('[가-힣0-9]+동 |[가-힣0-9]+동^[가-힣]|[가-힣0-9]+동[0-9]+|[가-힣0-9]+동$|[가-힣0-9]+리 |[가-힣0-9]+리[0-9]+|[가-힣0-9]+리^[가-힣]|[가-힣0-9]+리$')
s = '제주특별자치도 제주시 외도1동 482-1'
print(old_p.search(s))

<re.Match object; span=(12, 17), match='외도1동 '>


In [42]:
s = '경기도 시흥시 조남동'
print(old_p.search(s))

<re.Match object; span=(8, 11), match='조남동'>


In [43]:
s = '제주특별자치도 제주시 외도1동482-1'
print(old_p.search(s))

<re.Match object; span=(12, 19), match='외도1동482'>


In [44]:
s = '강원도 평창군 용평면 탑거리길 63 , 내고장 으뜸상품 판매장 앞'
print(old_p.search(s))

None


In [45]:
s = '전라북도 남원시 산내면 덕동리 406'
print(old_p.search(s))

<re.Match object; span=(13, 17), match='덕동리 '>


In [46]:
s = '전라북도 남원시 산내면 덕동리406'
print(old_p.search(s))

<re.Match object; span=(13, 19), match='덕동리406'>


In [47]:
s = '충청북도 보은군 마로면 적암리'
print(old_p.search(s))

<re.Match object; span=(13, 16), match='적암리'>


In [48]:
# 오류 주소
    # 신 : 충청북도 단양군 적성면 기동리
    # 신 : 제주특별자치도 제주시 외도1동 482-1
    # 구 : 강원도 평창군 용평면 탑거리길 63 , 내고장 으뜸상품 판매장 앞
    # 신 : 경기도 시흥시 조남동
    # 신 : 7755902
old_adr = []
old_idx = []
new_adr = []
new_idx = []
# new_p = re.compile('[가-힣]+로{1}|[가-힣]+로[0-9]+|[가-힣]+길|[가-힣0-9]+번길')
old_p = re.compile('[가-힣0-9]+동 |[가-힣0-9]+동^[가-힣]|[가-힣0-9]+동[0-9]+|[가-힣0-9]+동$|[가-힣0-9]+리 |[가-힣0-9]+리[0-9]+|[가-힣0-9]+리^[가-힣]|[가-힣0-9]+리$')
for i, s in tqdm(enumerate(temp)):
    # if new_p.search(s):
    if old_p.search(s):
        old_adr.append(s)
        old_idx.append(i)
    else:
        new_adr.append(s)
        new_idx.append(i)

print(len(new_idx))
print(len(old_idx))
print(len(temp))
print(len(new_idx) + len(old_idx) == len(temp))

11613449it [01:20, 144374.47it/s]

7755902
3857547
11613449
True


In [49]:
start_idx = 10*0
new_adr[start_idx:start_idx+10]

['경상남도 김해시 진영읍 하계로96번길 94-4 , 주유소 옆 주차장',
 '제주특별자치도 제주시 삼성로 40 민속자연사박물관',
 '제주특별자치도 제주시 노형9길 9-4,  주차장',
 '경상북도 포항시 북구 해안로82',
 '강원도 양양군 현북면 동해대로 1242',
 '제주특별자치도 제주시 산천단동길 26 인재개발원',
 '경상남도 양산시 양산대로 849 , 운동장 앞 도로쪽 주차장',
 '경상북도 칠곡군 동명면 중앙고속도로 123',
 '강원도 강릉시 강릉대로 33  P3 주차장',
 '제주특별자치도 제주시 우평로 318 부영아파트']

In [50]:
for i in new_adr:
    if '구로구' in i:
        print(i)
        break

서울특별시 구로구 경인로 661  지하2층 L


#### 빈칸 2개 -> 1개로 수정

In [51]:
# 빈칸 2개 있는것 1개로 변환
for i, s in enumerate(new_adr):
    if '  ' in s:
        new_adr[i] = s.replace('  ', ' ')

In [52]:
for i in new_adr:
    if '구로구' in i:
        print(i)
        break

서울특별시 구로구 경인로 661 지하2층 L


In [53]:
for i in new_adr:
    if '구로구 개봉동' in i:
        print(i)
        break

In [54]:
start_idx = 100
new_adr[start_idx:start_idx + 10]

['경기도 고양시 덕양구 고양시청로 10 , 주차장 입구 전 우측',
 '경상북도 상주시 화서면 터골길 51-25',
 '강원도 횡성군 안흥면 영동고속도로 154 , 휴게소와 LPG충전소 사이',
 '경상북도 청도군 청도읍 하지길 59-49 , 휴게소와 주유소 사이',
 '강원도 강릉시 강릉대로 33 P3 주차장',
 '강원도 횡성군 안흥면 영동고속도로 153 , 농축산물판매장 옆',
 '경기도 연천군 연천읍 연천로 220',
 '경상북도 영덕군 강구면 동해대로 4526',
 '경상북도 경산시 진량읍 대학로 1089 고분군 입구 우측 옥외 주차장',
 '충청북도 괴산군 장연면 중부내륙고속도로 205']

In [55]:
start_idx = 100
old_adr[start_idx:start_idx + 10]

['인천광역시 옹진군 영흥면 내리 8-165  공중화장실 옆',
 '제주특별자치도 서귀포시 성산읍 고성리 327-1',
 '제주특별자치도 서귀포시 안덕면 사계리 163-1 , 산방산 주차장 장애인 주차구역 앞',
 '경기도 안양시 동안구 관앙동 1590',
 '경상북도 군위군 산성면 백학리',
 '전라남도 여수시 소호동 505-2, 입구 우측',
 '제주특별자치도 서귀포시 서홍동 666-1',
 '강원도 양양군 현북면 하광정리 80-2',
 '세종특별자치시 나성남로 7-7 BRT환승센터 B주차장 B1 (나성동 531)',
 '강원도 양양군 현북면 하광정리 80-2']

#### 신주소, 구주소 데이터 프레임 분리

In [56]:
new_adr_df = df2.loc[new_idx].reset_index(drop=True)
old_adr_df = df2.loc[old_idx].reset_index(drop=True)

In [57]:
new_adr_df[(new_adr_df['지역'] != '서울특별시') & (new_adr_df['주소'] == '강남구 양재대로31길')]

,충전소명,충전기ID,지역,시군구,주소,충전기타입,충전시작일시,충전종료일시,충전시간,충전량,...,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산),raw값과_계산값_차이


In [58]:
id(df2), id(new_adr_df), id(old_adr_df)

(2300710790864, 2312404389072, 2302950987040)

In [59]:
new_adr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7755902 entries, 0 to 7755901
Data columns (total 27 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int8           
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기타입         object         
 6   충전시작일시        int64          
 7   충전종료일시        int64          
 8   충전시간          timedelta64[ns]
 9   충전량           float64        
 10  충전용량          object         
 11  충전시작_년도       int16          
 12  충전시작_월        int8           
 13  충전시작_일        int8           
 14  충전시작_시        int8           
 15  충전시작_분        int8           
 16  충전시작_초        int8           
 17  충전시작일시(변환)    object         
 18  충전종료_년도       int16          
 19  충전종료_월        int8           
 20  충전종료_일        int8           
 21  충전종료_시        int8           
 22  충전종료_분        int8           
 23  충전종료_초 

In [60]:
old_adr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3857547 entries, 0 to 3857546
Data columns (total 27 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int8           
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기타입         object         
 6   충전시작일시        int64          
 7   충전종료일시        int64          
 8   충전시간          timedelta64[ns]
 9   충전량           float64        
 10  충전용량          object         
 11  충전시작_년도       int16          
 12  충전시작_월        int8           
 13  충전시작_일        int8           
 14  충전시작_시        int8           
 15  충전시작_분        int8           
 16  충전시작_초        int8           
 17  충전시작일시(변환)    object         
 18  충전종료_년도       int16          
 19  충전종료_월        int8           
 20  충전종료_일        int8           
 21  충전종료_시        int8           
 22  충전종료_분        int8           
 23  충전종료_초 

### 신주소 정리
- 7755902 개

In [61]:
temp = new_adr_df['주소'].to_numpy()
temp[:10]

array(['경상남도 김해시 진영읍 하계로96번길 94-4 , 주유소 옆 주차장',
       '제주특별자치도 제주시 삼성로 40 민속자연사박물관', '제주특별자치도 제주시 노형9길 9-4,  주차장',
       '경상북도 포항시 북구 해안로82', '강원도 양양군 현북면 동해대로 1242',
       '제주특별자치도 제주시 산천단동길 26 인재개발원', '경상남도 양산시 양산대로 849 , 운동장 앞 도로쪽 주차장',
       '경상북도 칠곡군 동명면 중앙고속도로 123', '강원도 강릉시 강릉대로 33  P3 주차장',
       '제주특별자치도 제주시 우평로 318 부영아파트'], dtype=object)

In [62]:
id(temp), id(new_adr_df['주소']), id(new_adr_df)

(2301796696400, 2303271233376, 2312404389072)

#### 주소 표기 수정
- "전국 도로명 부여사유 조회(2022.08.31기준)" 파일 참고

##### 컴마(,) 제거

In [63]:
for i, s in enumerate(temp):
    if ',' in s:
        print(s)
        break

경상남도 김해시 진영읍 하계로96번길 94-4 , 주유소 옆 주차장


In [64]:
for i, s in enumerate(temp):
    if ',' in s:
        temp[i] = s.replace(',', '')

In [65]:
for i, s in enumerate(temp):
    if ',' in s:
        print(s)
        break

##### 도로명 형식에 맞게 수정

In [67]:
for i, one in enumerate(temp):
    if one == '광주광역시 북구 첨단 벤처로 108번길 9':
        print(i)
        break

880


In [68]:
temp[880]

'광주광역시 북구 첨단 벤처로 108번길 9'

In [69]:
'광주광역시 북구 첨단 벤처로 108번길 9'.replace('첨단 벤처로', '첨단벤처로')

'광주광역시 북구 첨단벤처로 108번길 9'

In [70]:
for i, one in enumerate(temp):
    if '첨단 벤처로' in one:
        temp[i] = one.replace('첨단 벤처로', '첨단벤처로')

In [71]:
temp[880]

'광주광역시 북구 첨단벤처로 108번길 9'

##### -길과 숫자 띄어쓰기

In [72]:
s = '전라북도 전주시 완산구 신촌3길24번지'
for i, one in enumerate(temp):
    if one == s:
        print(i)
        break

1525


In [73]:
temp[1525]

'전라북도 전주시 완산구 신촌3길24번지'

In [77]:
n_p = re.compile('([가-힣0-9]+길)([0-9]+)')
print(n_p.sub('\g<1> \g<2>', s))

전라북도 전주시 완산구 신촌3길 24번지


In [74]:
s = '전라남도 무안군 삼향읍 오룡1길 (E주차장-전라남도관광안내소)'
for i, one in enumerate(temp):
    if one == s:
        print(i)
        break

1145


In [78]:
temp[1145]

'전라남도 무안군 삼향읍 오룡1길 (E주차장-전라남도관광안내소)'

In [79]:
print(n_p.sub('\g<1> \g<2>', s))

전라남도 무안군 삼향읍 오룡1길 (E주차장-전라남도관광안내소)


In [80]:
# -길과 숫자 띄어쓰기
n_p = re.compile('([가-힣0-9]+길)([0-9]+)')

for i, one in enumerate(temp):
    if n_p.search(one):
        temp[i] = n_p.sub('\g<1> \g<2>', one)

In [81]:
temp[1525]

'전라북도 전주시 완산구 신촌3길 24번지'

In [82]:
temp[1145]

'전라남도 무안군 삼향읍 오룡1길 (E주차장-전라남도관광안내소)'

##### -로와 숫자 띄어쓰기

In [83]:
s = '경기도 이천시 마장면 중부고속도로82'
for i, one in enumerate(temp):
    if one == s:
        print(i)
        break

2195


In [84]:
temp[2195]

'경기도 이천시 마장면 중부고속도로82'

In [85]:
n_p = re.compile('([가-힣0-9]+로)([0-9]+)')
print(n_p.sub('\g<1> \g<2>', s))

경기도 이천시 마장면 중부고속도로 82


In [86]:
s = '강원도 강릉시 성산면 영동고속도로 232 (보광리)'
for i, one in enumerate(temp):
    if one == s:
        print(i)
        break

816


In [87]:
temp[816]

'강원도 강릉시 성산면 영동고속도로 232 (보광리)'

In [88]:
print(n_p.sub('\g<1> \g<2>', s))

강원도 강릉시 성산면 영동고속도로 232 (보광리)


In [89]:
# -로와 숫자 띄어쓰기
n_p = re.compile('([가-힣0-9]+로)([0-9]+)')

for i, one in enumerate(temp):
    if n_p.search(one):
        temp[i] = n_p.sub('\g<1> \g<2>', one)

In [90]:
temp[2195]

'경기도 이천시 마장면 중부고속도로 82'

In [91]:
temp[816]

'강원도 강릉시 성산면 영동고속도로 232 (보광리)'

##### -로-번-길 붙이기

In [92]:
s = '광주광역시 북구 첨단벤처로 108번길 9'
for i, one in enumerate(temp):
    if one == s:
        print(i)
        break

880


In [93]:
temp[880]

'광주광역시 북구 첨단벤처로 108번길 9'

In [94]:
n_p = re.compile('([가-힣0-9]+로) ([0-9]+번[가-힣]*길)')
print(n_p.search(s))
print(n_p.sub('\g<1>\g<2>', s))

<re.Match object; span=(9, 20), match='첨단벤처로 108번길'>
광주광역시 북구 첨단벤처로108번길 9


In [95]:
# -로-번-길 붙이기
n_p = re.compile('([가-힣0-9]+로) ([0-9]+번[가-힣]*길)')

for i, one in enumerate(temp):
    if n_p.search(one):
        temp[i] = n_p.sub('\g<1>\g<2>', one)

In [96]:
temp[880]

'광주광역시 북구 첨단벤처로108번길 9'

##### -로-길 붙이기

In [98]:
n_p = re.compile('([가-힣0-9]+로) ([0-9]+[가-힣]*길)')
for i, one in enumerate(temp):
    if n_p.search(one):
        print(i)
        break

40


In [100]:
temp[40]

'대구광역시 동구 효동로 2길 93'

In [101]:
s = '대구광역시 동구 효동로 2길 93'
print(n_p.search(s))
print(n_p.sub('\g<1>\g<2>', s))

<re.Match object; span=(9, 15), match='효동로 2길'>
대구광역시 동구 효동로2길 93


In [103]:
# -로-길 붙이기
n_p = re.compile('([가-힣0-9]+로) ([0-9]+[가-힣]*길)')

for i, one in enumerate(temp):
    if n_p.search(one):
        temp[i] = n_p.sub('\g<1>\g<2>', one)

In [104]:
temp[40]

'대구광역시 동구 효동로2길 93'

##### -앙-길 붙이기

In [106]:
n_p = re.compile('([가-힣0-9]+앙) ([0-9]+[가-힣]*길)')
for i, one in enumerate(temp):
    if n_p.search(one):
        print(i)
        break

In [108]:
# # -앙-길 붙이기
# n_p = re.compile('([가-힣0-9]+앙) ([0-9]+[가-힣]*길)')

# for i, one in enumerate(temp):
#     if n_p.search(one):
#         temp[i] = n_p.sub('\g<1>\g<2>', one)

##### -장-길 붙이기

In [107]:
n_p = re.compile('([가-힣0-9]+장) ([0-9]+[가-힣]*길)')
for i, one in enumerate(temp):
    if n_p.search(one):
        print(i)
        break

In [109]:
# # -장-길 붙이기
# n_p = re.compile('([가-힣0-9]+장) ([0-9]+[가-힣]*길)')

# for i, one in enumerate(temp):
#     if n_p.search(one):
#         temp[i] = n_p.sub('\g<1>\g<2>', one)

##### -솜-로 붙이기

In [110]:
n_p = re.compile('([가-힣0-9]+솜) ([0-9]+[가-힣]*로)')
for i, one in enumerate(temp):
    if n_p.search(one):
        print(i)
        break

1671


In [111]:
temp[1671]

'세종특별자치시 다솜 3로 66'

In [112]:
n_p = re.compile('([가-힣0-9]+솜) ([0-9]+[가-힣]*로)')
s = '세종특별자치시 다솜 3로'
print(n_p.search(s))
print(n_p.sub('\g<1>\g<2>', s))

<re.Match object; span=(8, 13), match='다솜 3로'>
세종특별자치시 다솜3로


In [113]:
n_p = re.compile('([가-힣0-9]+솜) ([0-9]+[가-힣]*로)')
s = '세종특별자치시 다솜3로'
print(n_p.search(s))
print(n_p.sub('\g<1>\g<2>', s))

None
세종특별자치시 다솜3로


In [114]:
# -솜-로 붙이기
n_p = re.compile('([가-힣0-9]+솜) ([0-9]+[가-힣]*로)')

for i, one in enumerate(temp):
    if n_p.search(one):
        temp[i] = n_p.sub('\g<1>\g<2>', one)

In [115]:
temp[1671]

'세종특별자치시 다솜3로 66'

#### 복수 주소 오류
- 한 주소에 2개 주소 들어있는 것 처리
- 동일지역 : 22,888개
- 다른지역 : 0개

##### 동일지역
- 22,888 개
- ex index(new_adr_df, temp) : 933745

In [122]:
same_local_idx = np.where(temp == '전라남도 무안군 망운면 운해로 1436-10 전라남도 함평군 신광면 학동로 1398-9')
same_local_idx[0][:5]

array([ 933745,  975149, 1384836, 1484838, 1535708], dtype=int64)

In [123]:
new_adr_df.loc[933745, '주소']

'전라남도 무안군 망운면 운해로 1436-10 전라남도 함평군 신광면 학동로 1398-9'

In [124]:
temp[933745]

'전라남도 무안군 망운면 운해로 1436-10 전라남도 함평군 신광면 학동로 1398-9'

In [125]:
'전라남도 무안군 망운면 운해로 1436-10 전라남도 함평군 신광면 학동로 1398-9'.replace(' 전라남도', '$전라남도').split('$')

['전라남도 무안군 망운면 운해로 1436-10', '전라남도 함평군 신광면 학동로 1398-9']

In [126]:
'전라남도 무안군 망운면 운해로 1436-10'.replace(' 전라남도', '$전라남도').split('$')

['전라남도 무안군 망운면 운해로 1436-10']

In [127]:
temp_si = new_adr_df['시군구'].to_numpy()
temp_si[:5]

array(['김해시', '제주시', '제주시', '포항시', '양양군'], dtype=object)

In [128]:
count = 0
for i, one in tqdm(enumerate(temp)):
    si = list(one.split())[0]
    
    one = one.replace(' ' + si, '$' + si)
    multi_adr = list(one.split('$'))
    if len(multi_adr) == 2:
        count += 1
    for j, two in enumerate(multi_adr):
        if temp_si[i] in two:
            temp[i] = multi_adr[j]

7755902it [00:22, 351278.86it/s]


In [129]:
temp[933745]

'전라남도 무안군 망운면 운해로 1436-10'

In [130]:
count

22888

##### 다른지역
- 0개

In [131]:
new_adr_df['지역'].unique()

array(['경상남도', '제주특별자치도', '경상북도', '강원도', '대전광역시', '충청남도', '충청북도', '부산광역시',
       '경기도', '전라남도', '대구광역시', '광주광역시', '서울특별시', '울산광역시', '전라북도', '인천광역시',
       '세종특별자치시'], dtype=object)

In [132]:
sep = ' 광주광역시| 전라남도| 전라북도| 제주특별자치도| 세종특별자치시| 서울특별시| 대전광역시| 경기도| 인천광역시| 경상남도| 강원도| 충청북도| 경상북도| 충청남도| 울산광역시| 대구광역시| 부산광역시'

In [133]:
re.split(sep, '광주광역시 북구')

['광주광역시 북구']

In [134]:
len(re.split(sep, '광주광역시 북구'))

1

In [135]:
re.split(sep, '광주광역시 충청북도')

['광주광역시', '']

In [136]:
len(re.split(sep, '광주광역시 충청북도'))

2

In [137]:
count = 0
for i, one in tqdm(enumerate(temp)):
    multi_local = list(re.split(sep, one))
    if len(multi_local) == 2:
        count += 1

7755902it [00:17, 432789.01it/s]


In [138]:
count

0

#### 신주소 정리 파일 기존 df(new_adr_df)로 복사

In [139]:
new_adr_df['주소'] = temp

In [140]:
id(new_adr_df['주소']), id(temp)

(2300710918464, 2301796696400)

In [141]:
new_adr_df['주소'].head()

0    경상남도 김해시 진영읍 하계로96번길 94-4  주유소 옆 주차장
1             제주특별자치도 제주시 삼성로 40 민속자연사박물관
2               제주특별자치도 제주시 노형9길 9-4  주차장
3                      경상북도 포항시 북구 해안로 82
4                   강원도 양양군 현북면 동해대로 1242
Name: 주소, dtype: object

In [142]:
# data : 7,755,902 개
new_adr_df['주소'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 7755902 entries, 0 to 7755901
Series name: 주소
Non-Null Count    Dtype 
--------------    ----- 
7755902 non-null  object
dtypes: object(1)
memory usage: 59.2+ MB


In [143]:
new_adr_df[(new_adr_df['지역'] != '서울특별시') & (new_adr_df['주소'] == '강남구 양재대로31길')]

,충전소명,충전기ID,지역,시군구,주소,충전기타입,충전시작일시,충전종료일시,충전시간,충전량,...,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산),raw값과_계산값_차이


### 구주소 정리
- 3,857,547 개

In [145]:
temp = old_adr_df['주소'].to_numpy()
temp[:10]

array(['서울특별시 구로구 오류동 174-10 , 천왕역 환승주차장 1층', '경상남도 통영시 광도면 죽림리 292-33',
       '전라북도 완주군 봉동읍 둔산리 881', '충청남도 공주시 정안면 석송리', '충청북도 단양군 적성면 기동리',
       '경상북도 영천시 임고면 금대리', '충청남도 예산군 예산읍 석양리 213-2',
       '경상남도 통영시 광도면 죽림리 292-33', '강원도 철원군 동송읍 장흥리 20-15',
       '경상북도 영천시 임고면 금대리'], dtype=object)

##### 컴마(,) 제거

In [146]:
for i, s in enumerate(temp):
    if ',' in s:
        print(s)
        break

서울특별시 구로구 오류동 174-10 , 천왕역 환승주차장 1층


In [147]:
for i, s in enumerate(temp):
    if ',' in s:
        temp[i] = s.replace(',', '')

In [148]:
temp[:10]

array(['서울특별시 구로구 오류동 174-10  천왕역 환승주차장 1층', '경상남도 통영시 광도면 죽림리 292-33',
       '전라북도 완주군 봉동읍 둔산리 881', '충청남도 공주시 정안면 석송리', '충청북도 단양군 적성면 기동리',
       '경상북도 영천시 임고면 금대리', '충청남도 예산군 예산읍 석양리 213-2',
       '경상남도 통영시 광도면 죽림리 292-33', '강원도 철원군 동송읍 장흥리 20-15',
       '경상북도 영천시 임고면 금대리'], dtype=object)

##### -동과 숫자 띄어쓰기

In [152]:
s = '제주특별자치도 제주시 연동2334-4'
for i, one in enumerate(temp):
    if one == s:
        print(i)
        break

988


In [150]:
temp[988]

'제주특별자치도 제주시 연동2334-4'

In [151]:
n_p = re.compile('([가-힣0-9]+동)([0-9]+)')
print(n_p.sub('\g<1> \g<2>', s))

제주특별자치도 제주시 연동 2334-4


In [153]:
s = '전라북도 정읍시 내장동 52'
for i, one in enumerate(temp):
    if one == s:
        print(i)
        break

447


In [154]:
temp[447]

'전라북도 정읍시 내장동 52'

In [155]:
print(n_p.sub('\g<1> \g<2>', s))

전라북도 정읍시 내장동 52


In [156]:
# -동과 숫자 띄어쓰기
n_p = re.compile('([가-힣0-9]+동)([0-9]+)')

for i, one in enumerate(temp):
    if n_p.search(one):
        temp[i] = n_p.sub('\g<1> \g<2>', one)

In [157]:
temp[988]

'제주특별자치도 제주시 연동 2334-4'

In [158]:
temp[447]

'전라북도 정읍시 내장동 52'

##### -리과 숫자 띄어쓰기

In [160]:
n_p = re.compile('([가-힣0-9]+리)([0-9]+)')
for i, one in enumerate(temp):
    if n_p.search(one):
        print(i)
        break

190


In [161]:
temp[190]

'전라북도 익산시 함열읍 와리164'

In [162]:
s = '전라북도 익산시 함열읍 와리164'
n_p = re.compile('([가-힣0-9]+리)([0-9]+)')
print(n_p.sub('\g<1> \g<2>', s))

전라북도 익산시 함열읍 와리 164


In [163]:
s = '충청북도 단양군 적성면 기동리'
for i, one in enumerate(temp):
    if one == s:
        print(i)
        break

4


In [164]:
temp[4]

'충청북도 단양군 적성면 기동리'

In [165]:
print(n_p.sub('\g<1> \g<2>', s))

충청북도 단양군 적성면 기동리


In [166]:
s = '충청북도 괴산군 괴산읍 서부리 654'
for i, one in enumerate(temp):
    if one == s:
        print(i)
        break

204


In [167]:
temp[204]

'충청북도 괴산군 괴산읍 서부리 654'

In [168]:
print(n_p.sub('\g<1> \g<2>', s))

충청북도 괴산군 괴산읍 서부리 654


In [169]:
# -리과 숫자 띄어쓰기
n_p = re.compile('([가-힣0-9]+리)([0-9]+)')

for i, one in enumerate(temp):
    if n_p.search(one):
        temp[i] = n_p.sub('\g<1> \g<2>', one)

In [170]:
temp[190]

'전라북도 익산시 함열읍 와리 164'

In [171]:
temp[4]

'충청북도 단양군 적성면 기동리'

In [172]:
temp[204]

'충청북도 괴산군 괴산읍 서부리 654'

#### 복수 주소 정리
- 한 주소에 2개 들어있는 것 처리

##### 동일 지역
- 99,756개

In [173]:
old_adr_df['주소'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 3857547 entries, 0 to 3857546
Series name: 주소
Non-Null Count    Dtype 
--------------    ----- 
3857547 non-null  object
dtypes: object(1)
memory usage: 29.4+ MB


In [174]:
for i, s in enumerate(old_adr_df['주소']):
    if ('청양군' in s) and ('태안군' in s):
        print(i)
        break

400516


In [175]:
old_adr_df['주소'].to_numpy()[400516]

'충청남도 청양군 청양읍 군량리 533 충청남도 태안군 소원면 신덕리 272'

In [176]:
np.where(temp == '충청남도 청양군 청양읍 군량리 533 충청남도 태안군 소원면 신덕리 272')[0][:5]

array([400516, 402168, 532375, 755429, 781579], dtype=int64)

In [177]:
temp[400516]

'충청남도 청양군 청양읍 군량리 533 충청남도 태안군 소원면 신덕리 272'

In [178]:
temp_si = old_adr_df['시군구'].to_numpy()
temp_si[:5]

array(['구로구', '통영시', '완주군', '공주시', '단양군'], dtype=object)

In [179]:
count = 0
for i, one in tqdm(enumerate(temp)):
    si = list(one.split())[0]
    one = one.replace(' ' + si, '$' + si)
    multi_adr = list(one.split('$'))
    if len(multi_adr) == 2:
        count += 1
    for j, two in enumerate(multi_adr):
        if temp_si[i] in two:
            temp[i] = multi_adr[j]

3857547it [00:11, 335577.14it/s]


In [180]:
temp[400516]

'충청남도 청양군 청양읍 군량리 533'

In [181]:
count

99756

##### 다른 지역

In [182]:
rep_dict = {" 광주광역시":"$광주광역시" ," 전라남도":"$전라남도"," 전라북도":"$전라북도"," 제주특별자치도":"$제주특별자치도"," 세종특별자치시":"$세종특별자치시"," 서울특별시":"$서울특별시"," 대전광역시":"$대전광역시"," 경기도":"$경기도"," 인천광역시":"$인천광역시"," 경상남도":"$경상남도"," 강원도":"$강원도"," 충청북도":"$충청북도"," 경상북도":"$경상북도"," 충청남도":"$충청남도"," 울산광역시":"$울산광역시"," 대구광역시":"$대구광역시"," 부산광역시":"$부산광역시"}

In [183]:
for k, v in rep_dict.items():
    print(k, v)

 광주광역시 $광주광역시
 전라남도 $전라남도
 전라북도 $전라북도
 제주특별자치도 $제주특별자치도
 세종특별자치시 $세종특별자치시
 서울특별시 $서울특별시
 대전광역시 $대전광역시
 경기도 $경기도
 인천광역시 $인천광역시
 경상남도 $경상남도
 강원도 $강원도
 충청북도 $충청북도
 경상북도 $경상북도
 충청남도 $충청남도
 울산광역시 $울산광역시
 대구광역시 $대구광역시
 부산광역시 $부산광역시


In [184]:
s = '전라북도 순창군 풍산면 반월리 12-4 충청남도 공주시 탄천면 삼각리 523-2'
for k, v in rep_dict.items():
    s = s.replace(k, v)
s

'전라북도 순창군 풍산면 반월리 12-4$충청남도 공주시 탄천면 삼각리 523-2'

In [186]:
s = '전라북도 순창군 풍산면 반월리 12-4 충청남도 공주시 탄천면 삼각리 523-2'
for i, one in enumerate(temp):
    if one == s:
        print(i)
        break

335492


In [187]:
temp[335492]

'전라북도 순창군 풍산면 반월리 12-4 충청남도 공주시 탄천면 삼각리 523-2'

In [188]:
temp[335552]

'전라북도 순창군 풍산면 반월리 12-4 충청남도 공주시 탄천면 삼각리 523-2'

In [161]:
sep = ' 광주광역시| 전라남도| 전라북도| 제주특별자치도| 세종특별자치시| 서울특별시| 대전광역시| 경기도| 인천광역시| 경상남도| 강원도| 충청북도| 경상북도| 충청남도| 울산광역시| 대구광역시| 부산광역시'

In [189]:
count = 0
multi_adr_old_indx = []
for i, one in enumerate(temp):
    multi_local = list(re.split(sep, one))
    if len(multi_local) == 2:
        multi_adr_old_indx.append(i)
        count += 1

In [190]:
count

5427

In [191]:
multi_adr_old_indx[:5]

[335492, 335552, 531411, 607542, 611395]

In [195]:
multi_adr_old_indx[-5:]

[3779709, 3804507, 3819507, 3840754, 3852186]

In [192]:
temp[335492]

'전라북도 순창군 풍산면 반월리 12-4 충청남도 공주시 탄천면 삼각리 523-2'

In [197]:
temp[3852186]

'전라북도 순창군 풍산면 반월리 12-4 충청남도 공주시 탄천면 삼각리 523-2'

In [198]:
count = 0
for i, one in tqdm(enumerate(temp)):
    for k, v in rep_dict.items():
        one = one.replace(k, v)
    multi_adr = list(one.split('$'))
    if len(multi_adr) == 2:
        count += 1
    for j, two in enumerate(multi_adr):
        if temp_si[i] in two:
            temp[i] = multi_adr[j]

3857547it [00:25, 148919.89it/s]


In [199]:
count

5427

In [200]:
temp[335492]

'전라북도 순창군 풍산면 반월리 12-4'

In [201]:
temp[3852186]

'전라북도 순창군 풍산면 반월리 12-4'

#### 구주소 정리 파일 기존 df(old_adr_df)로 복사

In [202]:
old_adr_df['주소'] = temp

In [203]:
old_adr_df['주소'].head()

0    서울특별시 구로구 오류동 174-10  천왕역 환승주차장 1층
1               경상남도 통영시 광도면 죽림리 292-33
2                  전라북도 완주군 봉동읍 둔산리 881
3                      충청남도 공주시 정안면 석송리
4                      충청북도 단양군 적성면 기동리
Name: 주소, dtype: object

In [204]:
# data : 3857547 개
old_adr_df['주소'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 3857547 entries, 0 to 3857546
Series name: 주소
Non-Null Count    Dtype 
--------------    ----- 
3857547 non-null  object
dtypes: object(1)
memory usage: 29.4+ MB


In [205]:
new_adr_df[(new_adr_df['지역'] != '서울특별시') & (new_adr_df['주소'] == '서울특별시 강남구 양재대로31길')]

,충전소명,충전기ID,지역,시군구,주소,충전기타입,충전시작일시,충전종료일시,충전시간,충전량,...,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산),raw값과_계산값_차이


##### 신주소, 구주소 분리 파일 출력

In [206]:
new_adr_df.to_csv('2020-22년_급속충전_신주소.csv', encoding='utf-8')

In [207]:
old_adr_df.to_csv('2020-22년_급속충전_구주소.csv', encoding='utf-8')

## 파일 로드(다시 시작)

In [4]:
# new_adr_df = pd.read_csv('2020-22년_급속충전_신주소.csv', index_col=0, dtype={'충전시작_년도':np.int16, '충전종료_년도':np.int16, '충전시작_월':np.int8,  '충전시작_일':np.int8, '충전시작_시':np.int8, '충전시작_분':np.int8, '충전시작_초':np.int8, '충전종료_월':np.int8, '충전종료_일':np.int8, '충전종료_시':np.int8, '충전종료_분':np.int8, '충전종료_초':np.int8, '충전기ID':np.int8})
# new_adr_df['충전시간'] = pd.to_timedelta(new_adr_df['충전시간'])
# new_adr_df['충전시간(계산)'] = pd.to_timedelta(new_adr_df['충전시간(계산)'])
# new_adr_df = new_adr_df.reset_index(drop=True)
# new_adr_df.info()

C:\Users\kbj\AppData\Local\Temp\ipykernel_13392\3769772588.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  new_adr_df = pd.read_csv('2020-22년_급속충전_신주소.csv', index_col=0, dtype={'충전시작_년도':np.int16, '충전종료_년도':np.int16, '충전시작_월':np.int8,  '충전시작_일':np.int8, '충전시작_시':np.int8, '충전시작_분':np.int8, '충전시작_초':np.int8, '충전종료_월':np.int8, '충전종료_일':np.int8, '충전종료_시':np.int8, '충전종료_분':np.int8, '충전종료_초':np.int8, '충전기ID':np.int8})


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7755902 entries, 0 to 7755901
Data columns (total 27 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int8           
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기타입         object         
 6   충전시작일시        int64          
 7   충전종료일시        int64          
 8   충전시간          timedelta64[ns]
 9   충전량           float64        
 10  충전용량          object         
 11  충전시작_년도       int16          
 12  충전시작_월        int8           
 13  충전시작_일        int8           
 14  충전시작_시        int8           
 15  충전시작_분        int8           
 16  충전시작_초        int8           
 17  충전시작일시(변환)    object         
 18  충전종료_년도       int16          
 19  충전종료_월        int8           
 20  충전종료_일        int8           
 21  충전종료_시        int8           
 22  충전종료_분        int8           
 23  충전종료_초 

In [5]:
# old_adr_df = pd.read_csv('2020-22년_급속충전_구주소.csv', index_col=0, dtype={'충전시작_년도':np.int16, '충전종료_년도':np.int16, '충전시작_월':np.int8,  '충전시작_일':np.int8, '충전시작_시':np.int8, '충전시작_분':np.int8, '충전시작_초':np.int8, '충전종료_월':np.int8, '충전종료_일':np.int8, '충전종료_시':np.int8, '충전종료_분':np.int8, '충전종료_초':np.int8, '충전기ID':np.int8})
# old_adr_df['충전시간'] = pd.to_timedelta(old_adr_df['충전시간'])
# old_adr_df['충전시간(계산)'] = pd.to_timedelta(old_adr_df['충전시간(계산)'])
# old_adr_df = old_adr_df.reset_index(drop=True)
# old_adr_df.info()

C:\Users\kbj\AppData\Local\Temp\ipykernel_13392\4119370516.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  old_adr_df = pd.read_csv('2020-22년_급속충전_구주소.csv', index_col=0, dtype={'충전시작_년도':np.int16, '충전종료_년도':np.int16, '충전시작_월':np.int8,  '충전시작_일':np.int8, '충전시작_시':np.int8, '충전시작_분':np.int8, '충전시작_초':np.int8, '충전종료_월':np.int8, '충전종료_일':np.int8, '충전종료_시':np.int8, '충전종료_분':np.int8, '충전종료_초':np.int8, '충전기ID':np.int8})


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3857547 entries, 0 to 3857546
Data columns (total 27 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int8           
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기타입         object         
 6   충전시작일시        int64          
 7   충전종료일시        int64          
 8   충전시간          timedelta64[ns]
 9   충전량           float64        
 10  충전용량          object         
 11  충전시작_년도       int16          
 12  충전시작_월        int8           
 13  충전시작_일        int8           
 14  충전시작_시        int8           
 15  충전시작_분        int8           
 16  충전시작_초        int8           
 17  충전시작일시(변환)    object         
 18  충전종료_년도       int16          
 19  충전종료_월        int8           
 20  충전종료_일        int8           
 21  충전종료_시        int8           
 22  충전종료_분        int8           
 23  충전종료_초 

In [6]:
# # [주의!]신/구주소 파일 불러올시 실행
# df = pd.concat([new_adr_df, old_adr_df], ignore_index=True)
# df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11613449 entries, 0 to 11613448
Data columns (total 27 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int8           
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기타입         object         
 6   충전시작일시        int64          
 7   충전종료일시        int64          
 8   충전시간          timedelta64[ns]
 9   충전량           float64        
 10  충전용량          object         
 11  충전시작_년도       int16          
 12  충전시작_월        int8           
 13  충전시작_일        int8           
 14  충전시작_시        int8           
 15  충전시작_분        int8           
 16  충전시작_초        int8           
 17  충전시작일시(변환)    object         
 18  충전종료_년도       int16          
 19  충전종료_월        int8           
 20  충전종료_일        int8           
 21  충전종료_시        int8           
 22  충전종료_분        int8           
 23  충전종료_

### 신주소 상세 분리

#### 상세 분리

In [7]:
new_adr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7755902 entries, 0 to 7755901
Data columns (total 27 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int8           
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기타입         object         
 6   충전시작일시        int64          
 7   충전종료일시        int64          
 8   충전시간          timedelta64[ns]
 9   충전량           float64        
 10  충전용량          object         
 11  충전시작_년도       int16          
 12  충전시작_월        int8           
 13  충전시작_일        int8           
 14  충전시작_시        int8           
 15  충전시작_분        int8           
 16  충전시작_초        int8           
 17  충전시작일시(변환)    object         
 18  충전종료_년도       int16          
 19  충전종료_월        int8           
 20  충전종료_일        int8           
 21  충전종료_시        int8           
 22  충전종료_분        int8           
 23  충전종료_초 

In [8]:
new_adr_df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기타입,충전시작일시,충전종료일시,충전시간,충전량,...,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산),raw값과_계산값_차이
0,진영휴게소(부산방향),1,경상남도,김해시,경상남도 김해시 진영읍 하계로96번길 94-4 주유소 옆 주차장,DC차데모+AC3상+DC콤보,20200101000244,20200101004215,0 days 00:39:25,20.22000,...,2020-01-01 00:02:44,2020,1,1,0,42,15,2020-01-01 00:42:15,0 days 00:39:31,0 days 00:00:06
1,민속자연사박물관,1,제주특별자치도,제주시,제주특별자치도 제주시 삼성로 40 민속자연사박물관,DC차데모+AC3상+DC콤보,20200101000745,20200101003703,0 days 00:29:13,16.04000,...,2020-01-01 00:07:45,2020,1,1,0,37,3,2020-01-01 00:37:03,0 days 00:29:18,0 days 00:00:05
2,노형동주민센터,1,제주특별자치도,제주시,제주특별자치도 제주시 노형9길 9-4 주차장,DC차데모+AC3상+DC콤보,20200101000748,20200101001659,0 days 00:09:07,5.80000,...,2020-01-01 00:07:48,2020,1,1,0,16,59,2020-01-01 00:16:59,0 days 00:09:11,0 days 00:00:04
3,영일대1주차장,1,경상북도,포항시,경상북도 포항시 북구 해안로 82,DC차데모+AC3상+DC콤보,20200101000851,20200101004220,0 days 00:33:29,13.32000,...,2020-01-01 00:08:51,2020,1,1,0,42,20,2020-01-01 00:42:20,0 days 00:33:29,0 days 00:00:00
4,38선 휴게소,1,강원도,양양군,강원도 양양군 현북면 동해대로 1242,DC차데모+AC3상+DC콤보,20200101001042,20200101005123,0 days 00:40:41,29.23000,...,2020-01-01 00:10:42,2020,1,1,0,51,23,2020-01-01 00:51:23,0 days 00:40:41,0 days 00:00:00


In [9]:
temp = new_adr_df['주소'].str.split(expand=True)
temp.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,경상남도,김해시,진영읍,하계로96번길,94-4,주유소,옆,주차장,None,None,None,None,None
1,제주특별자치도,제주시,삼성로,40,민속자연사박물관,None,None,None,None,None,None,None,None
2,제주특별자치도,제주시,노형9길,9-4,주차장,None,None,None,None,None,None,None,None
3,경상북도,포항시,북구,해안로,82,None,None,None,None,None,None,None,None
4,강원도,양양군,현북면,동해대로,1242,None,None,None,None,None,None,None,None


In [10]:
temp.shape

(7755902, 13)

In [11]:
# 5번컬럼까지 -길, -로, -번길 존재
# 6번부터 없음
check_p = re.compile('로$|길$|번길$')
for one in range(5, 13):
    print(f'{one}', end=' : ')
    uni = temp[temp[one].notnull() == True][one].unique()
    # print(uni)
    count = 0
    for two in uni:
        if check_p.search(two):
            count += 1
            break
    if count != 0:
        print('Yes')
    else:
        print('No')

5 : Yes
6 : No
7 : No
8 : No
9 : No
10 : No
11 : No
12 : No


In [12]:
temp[5].unique()

array(['주유소', None, '앞', '주차장', '행담도휴게소(목포방향)', '휴게소', '289', '동부소방서',
       '휴게소와', '본관', '농축산물판매장', '대형주차장', '1층', '제2청사)', '건물', '출연자',
       '성남시청', '4층', '행담도휴게소(서울방향)', '서생면사무소', '(보광리)', '공중화장실', '(방목리)',
       'D15', '309', '(오가리)', '성판악휴게소', '(농협)', '(제비리)', '(방내리)', '13',
       '고분군', '(성산리)', '알뜰휴게소', '보성군청', '(용전리)', '793-40', '출구', '(강학리)',
       '(옥산리)', '애월읍도서관', '(죽전동)', '농산물매장', '1511', '(성하리)', '(양곡리)',
       '(송계리)', '주차', '(보산리)', '민원봉사실', '136', '가평휴게소', '왼쪽', '407', '우측',
       '(도리)', '(율암리)', '율동자연공원', '알펜시아', '(신양리)', '옆', '1F)', '좌측',
       '마리나요트클럽', '23-37', '47', '(공세동)', '후문', '(순지리)', '매표소', '추자면사무소',
       '2번', '(외부리)', '소형주차장', '자치경찰대', '후포문화복지센터', '75', '뒤편', '영암군청',
       '(망상동)', '지상주차장', '(두대동)', '경비실', '제주농업기술센터', '내고장', '(회포리)', '05',
       '표선생활체육관', '환경성질환예방관리센터', '옥외주차장', '(삼룡동)173-1', '입구', '장흥군청',
       '576', '20', '(고림동)', '(우천리)', '236-1', '출구방향', '2층', '(가천리)',
       '수원도시공사', '화장실', '(쌍용2동)', '끝', '(주수리)', '구좌체육관', '(유현리)',

In [13]:
temp[temp[5] == '온달평강3로'].head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
637957,충청북도,단양군,영춘면,온달평강3로,9,온달평강3로,9-1,None,None,None,None,None,None
637997,충청북도,단양군,영춘면,온달평강3로,9,온달평강3로,9-1,None,None,None,None,None,None
641965,충청북도,단양군,영춘면,온달평강3로,9,온달평강3로,9-1,None,None,None,None,None,None
646183,충청북도,단양군,영춘면,온달평강3로,9,온달평강3로,9-1,None,None,None,None,None,None
652948,충청북도,단양군,영춘면,온달평강3로,9,온달평강3로,9-1,None,None,None,None,None,None


In [185]:
temp[temp[5] == '온달평강3로'].shape

(808, 13)

In [186]:
temp[4].unique()

array(['94-4', '민속자연사박물관', '주차장', ..., '21-8', '941', '247'], dtype=object)

In [187]:
temp = temp.fillna(np.nan)

In [188]:
temp.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,경상남도,김해시,진영읍,하계로96번길,94-4,주유소,옆,주차장,NaN,NaN,NaN,NaN,NaN
1,제주특별자치도,제주시,삼성로,40,민속자연사박물관,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,제주특별자치도,제주시,노형9길,9-4,주차장,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,경상북도,포항시,북구,해안로,82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,강원도,양양군,현북면,동해대로,1242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
temp.shape

(7755902, 13)

In [190]:
new_adr_df2 = pd.concat([new_adr_df, temp], axis=1)
new_adr_df2.head()

,충전소명,충전기ID,지역,시군구,주소,충전기타입,충전시작일시,충전종료일시,충전시간,충전량,...,3,4,5,6,7,8,9,10,11,12
0,진영휴게소(부산방향),1,경상남도,김해시,경상남도 김해시 진영읍 하계로96번길 94-4 주유소 옆 주차장,DC차데모+AC3상+DC콤보,20200101000244,20200101004215,0 days 00:39:25,20.22,...,하계로96번길,94-4,주유소,옆,주차장,NaN,NaN,NaN,NaN,NaN
1,민속자연사박물관,1,제주특별자치도,제주시,제주특별자치도 제주시 삼성로 40 민속자연사박물관,DC차데모+AC3상+DC콤보,20200101000745,20200101003703,0 days 00:29:13,16.04,...,40,민속자연사박물관,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,노형동주민센터,1,제주특별자치도,제주시,제주특별자치도 제주시 노형9길 9-4 주차장,DC차데모+AC3상+DC콤보,20200101000748,20200101001659,0 days 00:09:07,5.80,...,9-4,주차장,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,영일대1주차장,1,경상북도,포항시,경상북도 포항시 북구 해안로 82,DC차데모+AC3상+DC콤보,20200101000851,20200101004220,0 days 00:33:29,13.32,...,해안로,82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,38선 휴게소,1,강원도,양양군,강원도 양양군 현북면 동해대로 1242,DC차데모+AC3상+DC콤보,20200101001042,20200101005123,0 days 00:40:41,29.23,...,동해대로,1242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [191]:
new_adr_df2[(new_adr_df2['지역'] != '서울특별시') & (new_adr_df2['주소'] == '강남구 양재대로31길')]

,충전소명,충전기ID,지역,시군구,주소,충전기타입,충전시작일시,충전종료일시,충전시간,충전량,...,3,4,5,6,7,8,9,10,11,12


In [192]:
new_adr_df2.shape

(7755902, 40)

In [328]:
# new_adr_df2 = pd.concat([new_adr_df, temp], axis=1, ignore_index=True)
# new_adr_df2.head()

##### 신주소 상세 분리 확인

In [193]:
new_adr_df2[['주소', 0, 1, 2, 3, 4]].tail()

,주소,0,1,2,3,4
7755897,경기도 구리시 체육관로 74,경기도,구리시,체육관로,74,NaN
7755898,서울특별시 성동구 둘레길 47-5,서울특별시,성동구,둘레길,47-5,NaN
7755899,경기도 용인시 수지구 경부고속도로 400 (죽전동),경기도,용인시,수지구,경부고속도로,400
7755900,강원도 원주시 단구로 150,강원도,원주시,단구로,150,NaN
7755901,서울특별시 송파구 성내천로 299-7,서울특별시,송파구,성내천로,299-7,NaN


In [194]:
new_adr_df2[['주소', 0, 1, 2, 3, 4]].head()

,주소,0,1,2,3,4
0,경상남도 김해시 진영읍 하계로96번길 94-4 주유소 옆 주차장,경상남도,김해시,진영읍,하계로96번길,94-4
1,제주특별자치도 제주시 삼성로 40 민속자연사박물관,제주특별자치도,제주시,삼성로,40,민속자연사박물관
2,제주특별자치도 제주시 노형9길 9-4 주차장,제주특별자치도,제주시,노형9길,9-4,주차장
3,경상북도 포항시 북구 해안로 82,경상북도,포항시,북구,해안로,82
4,강원도 양양군 현북면 동해대로 1242,강원도,양양군,현북면,동해대로,1242


##### 신주소 상세 분리 파일 출력

In [195]:
new_adr_df2.to_csv('2020-22년_급속충전_신주소_상세분리.csv', encoding='utf-8')

### 구주소 상세분리

#### 상세분리

In [196]:
temp = old_adr_df['주소'].str.split(expand=True)
temp.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,서울특별시,구로구,오류동,174-10,천왕역,환승주차장,1층,None,None,None,None,None
1,경상남도,통영시,광도면,죽림리,292-33,None,None,None,None,None,None,None
2,전라북도,완주군,봉동읍,둔산리,881,None,None,None,None,None,None,None
3,충청남도,공주시,정안면,석송리,None,None,None,None,None,None,None,None
4,충청북도,단양군,적성면,기동리,None,None,None,None,None,None,None,None


In [197]:
# 5번컬럼까지 -동, -리 존재
# 6번부터 없음
check_p = re.compile('동$|리$')
for one in range(4, 12):
    print(f'{one}', end=' : ')
    uni = temp[temp[one].notnull() == True][one].unique()
    # print(uni)
    count = 0
    for two in uni:
        if check_p.search(two):
            count += 1
            break
    if count != 0:
        print('Yes')
    else:
        print('No')

4 : Yes
5 : Yes
6 : Yes
7 : Yes
8 : No
9 : Yes
10 : No
11 : No


In [198]:
temp[7].unique()

array([None, '옆', '입구', '장애인', '531)', '사이', '18', '주차장', 'E01번', '공영주차장',
       '정면', '주민센터', '앞', '좌측', '우측', '근처', '왼쪽', '대형주차장', '(묵현리', '내',
       '기둥옆', '오송역', '제2공구', '오른쪽', '제2주차장'], dtype=object)

In [199]:
temp[6].unique()

array(['1층', None, '오른쪽', '앞', '끝', '버스주차장', '사무실', '우측', '옆', '주차장',
       '(나성동', '뒤', '관리사무소', 'LPG충전소', '지하4층', '근처', '증설주차장', '입구', '2층',
       '맨끝쪽', '맞은편', '정면', '좌측', '2동', '건물입구', '건물', '후면', '지하주차장', '3가)',
       '흡연실', '1282-16)', '맨끝', '384-27)', '왼쪽', '20구역', '287기둥옆',
       '(기동리)', '430기둥', '820-10)', '사업단', '공영주차장', '밑', '5층', '안쪽', '9',
       '산업단지', '매표소앞', '가03', '주차구역', 'KTX', '마금산온천', '옥상', '501)', '1가)',
       '2', '232-9'], dtype=object)

In [200]:
temp[5].unique()

array(['환승주차장', None, '입구', '주차장', '좌측', '주차타워', '제2공영주차장', '안쪽', '우측',
       '북구청', '지상', '뒤편', '주민센터', '공중화장실', '산방산', 'B1', '485', '온정종합터미널',
       '정문', '휴게소와', '무릉문화의집', '179-6', '15', '검품장', '북측', '끝', '제주현대미술관',
       '지하', '새마을금고', '필지', '2층주차장', '28', '44', '재릉종합관광안내센터', '웅동',
       '20-48', '구청', '고엽제전우회', '753', '499-1)', '군청', '4층매장입구옆', '번지',
       '옆', '후생동', '14', '정면', '604-3', '(창포동', '본관', '(백운리', '별관',
       '630-5', '마트입구', '1712-10', '(여양3로', '1층', '지상2층', '807-6', '건너편',
       '32-37', '그린빌딩', '230-1', '2-15', '건물', '(신관리', '스포츠', '97-1)',
       '사랑방', '천마산군립공원', '보조경기장', '골프장', '지상주차장', '24-3', '(향교동)', '일원',
       '1234-3', '입구앞', '1길', '대마전기차', '182-131', '영암국제카트장', '지하1층', '대형',
       '앞', '소형주차장', '26', '370-1', '북주차장(우측)', '403-3', '386-2', '407-1',
       '인근', '16-1', '공영주차장', '3층', '천의삼거리', '307', '312', '57-26',
       '954-4', '292-5', '30-2', '679', '1316', '1257', '7-10', '664-5',
       '민원봉사실', '42', '11', '391-3', '30-1', '353', '376-1', 

In [201]:
temp = temp.fillna(np.nan)

In [202]:
temp.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,서울특별시,구로구,오류동,174-10,천왕역,환승주차장,1층,NaN,NaN,NaN,NaN,NaN
1,경상남도,통영시,광도면,죽림리,292-33,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,전라북도,완주군,봉동읍,둔산리,881,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,충청남도,공주시,정안면,석송리,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,충청북도,단양군,적성면,기동리,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [203]:
old_adr_df2 = pd.concat([old_adr_df, temp], axis=1)
old_adr_df2.head()

,충전소명,충전기ID,지역,시군구,주소,충전기타입,충전시작일시,충전종료일시,충전시간,충전량,...,2,3,4,5,6,7,8,9,10,11
0,천왕역 환승주차장,1,서울특별시,구로구,서울특별시 구로구 오류동 174-10 천왕역 환승주차장 1층,DC차데모+AC3상,20200101000457,20200101004534,0 days 00:40:37,18.87,...,오류동,174-10,천왕역,환승주차장,1층,NaN,NaN,NaN,NaN,NaN
1,신세계종합정비,1,경상남도,통영시,경상남도 통영시 광도면 죽림리 292-33,DC차데모+AC3상+DC콤보,20200101000906,20200101004945,0 days 00:40:39,27.96,...,광도면,죽림리,292-33,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,근로자종합복지관,1,전라북도,완주군,전라북도 완주군 봉동읍 둔산리 881,DC차데모+AC3상+DC콤보,20200101001519,20200101005604,0 days 00:40:45,25.64,...,봉동읍,둔산리,881,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,정안휴게소(순천방향),1,충청남도,공주시,충청남도 공주시 정안면 석송리,DC차데모+AC3상+DC콤보,20200101002014,20200101002652,0 days 00:06:38,1.17,...,정안면,석송리,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,단양휴게소(부산방향),1,충청북도,단양군,충청북도 단양군 적성면 기동리,DC차데모+AC3상+DC콤보,20200101003527,20200101004937,0 days 00:14:06,8.64,...,적성면,기동리,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### 주소 상세 분리 확인

In [204]:
old_adr_df2[['주소', 0, 1, 2, 3, 4]].tail()

,주소,0,1,2,3,4
3857542,경기도 안산시 상록구 사동 1271-11,경기도,안산시,상록구,사동,1271-11
3857543,경기도 의정부시 호원동 229-160,경기도,의정부시,호원동,229-160,NaN
3857544,서울특별시 서초구 반포동 49-8,서울특별시,서초구,반포동,49-8,NaN
3857545,인천광역시 서구 당하동 1097-5 검단4동 주민센터 옆,인천광역시,서구,당하동,1097-5,검단4동
3857546,대구광역시 남구 봉덕동 1272-18,대구광역시,남구,봉덕동,1272-18,NaN


In [205]:
old_adr_df2[['주소', 0, 1, 2, 3, 4]].head()

,주소,0,1,2,3,4
0,서울특별시 구로구 오류동 174-10 천왕역 환승주차장 1층,서울특별시,구로구,오류동,174-10,천왕역
1,경상남도 통영시 광도면 죽림리 292-33,경상남도,통영시,광도면,죽림리,292-33
2,전라북도 완주군 봉동읍 둔산리 881,전라북도,완주군,봉동읍,둔산리,881
3,충청남도 공주시 정안면 석송리,충청남도,공주시,정안면,석송리,NaN
4,충청북도 단양군 적성면 기동리,충청북도,단양군,적성면,기동리,NaN


##### 구주소 상세 분리 파일 출력

In [206]:
old_adr_df2.to_csv('2020-22년_급속충전_구주소_상세분리.csv', encoding='utf-8')

In [207]:
for one in old_adr_df2['주소'].to_list():
    if ('제주' in one):
        print(one)
        break

제주특별자치도 서귀포시 성산읍 성산리 347-9 주차타워 앞


## 파일 로드(다시 시작)

In [208]:
# new_adr_df2 = pd.read_csv('2020-22년_급속충전_신주소_상세분리.csv', index_col=0, dtype={'충전시작_년도':np.int16, '충전종료_년도':np.int16, '충전시작_월':np.int8,  '충전시작_일':np.int8, '충전시작_시':np.int8, '충전시작_분':np.int8, '충전시작_초':np.int8, '충전종료_월':np.int8, '충전종료_일':np.int8, '충전종료_시':np.int8, '충전종료_분':np.int8, '충전종료_초':np.int8, '충전기ID':np.int8})
# new_adr_df2['충전시간'] = pd.to_timedelta(new_adr_df2['충전시간'])
# new_adr_df2['충전시간(계산)'] = pd.to_timedelta(new_adr_df2['충전시간(계산)'])
# new_adr_df2 = new_adr_df2.reset_index(drop=True)
# new_adr_df2.info()

In [209]:
# old_adr_df2 = pd.read_csv('2020-22년_급속충전_구주소_상세분리.csv', index_col=0, dtype={'충전시작_년도':np.int16, '충전종료_년도':np.int16, '충전시작_월':np.int8,  '충전시작_일':np.int8, '충전시작_시':np.int8, '충전시작_분':np.int8, '충전시작_초':np.int8, '충전종료_월':np.int8, '충전종료_일':np.int8, '충전종료_시':np.int8, '충전종료_분':np.int8, '충전종료_초':np.int8, '충전기ID':np.int8})
# old_adr_df2['충전시간'] = pd.to_timedelta(old_adr_df2['충전시간'])
# old_adr_df2['충전시간(계산)'] = pd.to_timedelta(old_adr_df2['충전시간(계산)'])
# old_adr_df2 = old_adr_df2.reset_index(drop=True)
# old_adr_df2.info()

In [210]:
# # [주의!]신/구주소 파일 불러올시 실행
# df2 = pd.concat([new_adr_df2, old_adr_df2], ignore_index=True)
# df2.info()

## 분석2

In [211]:
new_adr_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7755902 entries, 0 to 7755901
Data columns (total 40 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int8           
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기타입         object         
 6   충전시작일시        int64          
 7   충전종료일시        int64          
 8   충전시간          timedelta64[ns]
 9   충전량           float64        
 10  충전용량          object         
 11  충전시작_년도       int16          
 12  충전시작_월        int8           
 13  충전시작_일        int8           
 14  충전시작_시        int8           
 15  충전시작_분        int8           
 16  충전시작_초        int8           
 17  충전시작일시(변환)    object         
 18  충전종료_년도       int16          
 19  충전종료_월        int8           
 20  충전종료_일        int8           
 21  충전종료_시        int8           
 22  충전종료_분        int8           
 23  충전종료_초 

In [212]:
old_adr_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3857547 entries, 0 to 3857546
Data columns (total 39 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int8           
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기타입         object         
 6   충전시작일시        int64          
 7   충전종료일시        int64          
 8   충전시간          timedelta64[ns]
 9   충전량           float64        
 10  충전용량          object         
 11  충전시작_년도       int16          
 12  충전시작_월        int8           
 13  충전시작_일        int8           
 14  충전시작_시        int8           
 15  충전시작_분        int8           
 16  충전시작_초        int8           
 17  충전시작일시(변환)    object         
 18  충전종료_년도       int16          
 19  충전종료_월        int8           
 20  충전종료_일        int8           
 21  충전종료_시        int8           
 22  충전종료_분        int8           
 23  충전종료_초 

#### 분석 이슈
- 1번 지역(시군구) : 세종특별자치시 주소는 시군구가 없이 바로 도로명 주소나 2번 지역명(읍면동)이 있는 경우가 있음
- 2번 지역(읍면동) : 3번 지역명(리)의 주소가 있는 샘플 존재.
    - 샘플 주소 : 강원도 화천군 하리
    - 실제 주소 : 강원도 화천군 화천읍 하리

#### 이슈 해결 방안
- 1\. 신주소, 구주소 나누어 분석
- 2\. 주소에서 빠진 지역구분명은 검색을 통해 주소 값 입력
- 3\. 해당 주소 다시 지역구분으로 분리

### 지역별 유니크 값 확인
- 지역 분류에 맞는 주소 들어갔는지 확인

In [13]:
n = 1
for one in list(df2['2'].unique()):
    if n % 10 == 0:
        print(one)
    else:
        print(one, end=', ')
    n += 1

진영읍, 삼성로, 노형9길, 북구, 현북면, 산천단동길, 양산대로, 동명면, 강릉대로, 우평로
월드컵대로, 용평면, 신평면, 애월읍, 문막읍, 서원구, 강구면, 성산읍, 안흥면, 장안읍
송산면, 문내면, 안면읍, 선원로, 봉개북1길, 문연로, 효동로2길, 일산서구, 남문로, 청도읍
화서면, 남구, 지양로, 봉수로, 기자길, 광양읍, 오전로, 서광로2길, 단원구, 팔달로
중원구, 동호로, 세종대로, 서생면, 지석13길, 설악면, 성산면, 단성면, 북한강변길, 흥덕구
장연면, 조천읍, 칠서면, 구정면, 건천읍, 마산합포구, 덕양구, 연천읍, 진량읍, 안성면
비인면, 내산면, 청석로3길, 팔탄면, 낙성대로4가길, 농소면, 보성읍, 선소로, 중앙탑면, 옥천읍
동남구, 중문관광로, 부발읍, 하신중앙로, 와촌면, 능주면, 선덕로8길, 호저면, 망운면, 수지구
죽변면, 옥성면, 현풍면, 문산읍, 안계면, 송지면, 분당구, 청송읍, 대가면, 죽왕면
수목원길, 계룡로, 임고면, 돈내코로, 신림면, 지곡면, 구지면, 내토로, 동홍로, 왜관읍
율촌면, 이월면, 현충로, 의사당대로, 장안로, 연희로, 서북구, 동진5길, 중앙로, 중부내륙고속도로
표선면, 진월면, 한림읍, 가조면, 북성로, 관아문길, 양남면, 함평읍, 삼동길, 군위읍
대관령면, 벌곡면, 무학로, 추암길, 영중로, 소보리당로164번길, 장충단로, 평원로, 큰발개, 해안마을서2길
남승룡로, 미시령로, 기흥구, 해미면, 원효로35길, 화양읍, 하북면, 남성중로, 추자면, 봉오대로
중부고속도로, 돌산읍, 해운대로, 군북면, 석문면, 보목로64번길, 서면, 강현면, 석봉로38번길, 부두로
용현면, 후포면, 무왕로, 정안면, 새천년도로, 합천읍, 병곡면, 퇴계로, 성주읍, 부곡로
근덕면, 산척면, 영암읍, 봉은사로, 오남로, 으뜸로, 중앙대로, 대강면, 한경면, 동해대로
화도읍, 첨단로, 동문대로, 가정북로, 의창구, 고창읍, 매송면, 가정로, 혁신6로, 문화회관4길
이동면, 남매로, 계북면, 영동읍, 황간면, 망우로, 구이면

### 신주소 분석

In [214]:
new_adr_df2[0].unique()

array(['경상남도', '제주특별자치도', '경상북도', '강원도', '대전광역시', '충청남도', '충청북도', '부산광역시',
       '경기도', '전라남도', '대구광역시', '광주광역시', '서울특별시', '울산광역시', '전라북도', '인천광역시',
       '세종특별자치시'], dtype=object)

In [215]:
new_adr_df2[1].unique()

array(['김해시', '제주시', '포항시', '양양군', '양산시', '칠곡군', '강릉시', '유성구', '평창군',
       '당진시', '원주시', '청주시', '영덕군', '서귀포시', '횡성군', '기장군', '화성시', '해남군',
       '태안군', '달서구', '동구', '고양시', '청도군', '상주시', '양천구', '광양시', '의왕시',
       '안산시', '북구', '성남시', '중구', '종로구', '울주군', '가평군', '산청군', '춘천시', '괴산군',
       '함안군', '경주시', '창원시', '연천군', '경산시', '무주군', '서천군', '부여군', '삼척시',
       '관악구', '김천시', '보성군', '여수시', '충주시', '옥천군', '천안시', '이천시', '사하구',
       '화순군', '무안군', '용인시', '울진군', '구미시', '달성군', '진주시', '의성군', '청송군',
       '고성군', '거제시', '영천시', '함양군', '제천시', '진천군', '동작구', '영등포구', '계룡시',
       '서대문구', '문경시', '거창군', '서산시', '함평군', '군위군', '논산시', '수성구', '동해시',
       '통영시', '순천시', '속초시', '용산구', '계양구', '하남시', '해운대구', '금산군', '대덕구',
       '사천시', '익산시', '공주시', '합천군', '안동시', '성주군', '영암군', '강남구', '동래구',
       '단양군', '남양주시', '고창군', '남해군', '군산시', '장수군', '영동군', '중랑구', '완주군',
       '부산진구', '홍성군', '포천시', '강진군', '광주시', '광산구', '서구', '장흥군', '사상구',
       '홍천군', '영광군', '서초구', '남구', '강서구', '순창군', '고령군', '강동구', '부평구',
       '수원시'

In [217]:
sort_ad2 = re.compile('[가-힣]+시$|[가-힣]+군$|[가-힣]+구$')
strange_ad2s = []
for one in new_adr_df2[1].unique():
    if sort_ad2.search(one) == None:
        strange_ad2s.append(one)
        print(one)

한누리대로
다솜3로
도움6로
연기면
다솜2로
가름로
보듬6로
정부2청사로
도움5로
갈매로
국세청로
다솜로
조치원읍
장군면
금남면


In [218]:
for one in strange_ad2s:
    print(f'지역2 : {one}')
    print(new_adr_df2[new_adr_df2[1] == one][0].unique())
    print(new_adr_df2[new_adr_df2[1] == one][1].unique())
    print(new_adr_df2[new_adr_df2[1] == one][2].unique())
    print()

지역2 : 한누리대로
['세종특별자치시']
['한누리대로']
['402']

지역2 : 다솜3로
['세종특별자치시']
['다솜3로']
['66' '95(어진동)']

지역2 : 도움6로
['세종특별자치시']
['도움6로']
['11']

지역2 : 연기면
['세종특별자치시']
['연기면']
['정안세종로' '수목원로']

지역2 : 다솜2로
['세종특별자치시']
['다솜2로']
['11' '94(어진동)']

지역2 : 가름로
['세종특별자치시']
['가름로']
['238-1']

지역2 : 보듬6로
['세종특별자치시']
['보듬6로']
['16']

지역2 : 정부2청사로
['세종특별자치시']
['정부2청사로']
['13(나성동)']

지역2 : 도움5로
['세종특별자치시']
['도움5로']
['20']

지역2 : 갈매로
['세종특별자치시']
['갈매로']
['477' '408(어진동)' '388(어진동)']

지역2 : 국세청로
['세종특별자치시']
['국세청로']
['8-14(나성동)']

지역2 : 다솜로
['세종특별자치시']
['다솜로']
['261']

지역2 : 조치원읍
['세종특별자치시']
['조치원읍']
['조치원6길']

지역2 : 장군면
['세종특별자치시']
['장군면']
['정자말길']

지역2 : 금남면
['세종특별자치시']
['금남면']
['산림박물관길']



In [18]:
new_adr_df2[new_adr_df2['2'] == '진영읍'].head()

,충전소명,충전기ID,지역,시군구,주소,충전기타입,충전시작일시,충전종료일시,충전시간,충전량,...,3,4,5,6,7,8,9,10,11,12
0,진영휴게소(부산방향),1,경상남도,김해시,경상남도 김해시 진영읍 하계로96번길 94-4 주유소 옆 주차장,DC차데모+AC3상+DC콤보,20200101000244,20200101004215,0 days 00:39:25,20.22,...,하계로96번길,94-4,주유소,옆,주차장,NaN,NaN,NaN,NaN,NaN
126,진영휴게소(부산방향),1,경상남도,김해시,경상남도 김해시 진영읍 하계로96번길 94-4 주유소 옆 주차장,DC차데모+AC3상+DC콤보,20200101035452,20200101043311,0 days 00:38:14,20.43,...,하계로96번길,94-4,주유소,옆,주차장,NaN,NaN,NaN,NaN,NaN
172,진영휴게소(부산방향),1,경상남도,김해시,경상남도 김해시 진영읍 하계로96번길 94-4 주유소 옆 주차장,DC차데모+AC3상+DC콤보,20200101044506,20200101045533,0 days 00:10:23,6.01,...,하계로96번길,94-4,주유소,옆,주차장,NaN,NaN,NaN,NaN,NaN
189,진영휴게소(부산방향),1,경상남도,김해시,경상남도 김해시 진영읍 하계로96번길 94-4 주유소 옆 주차장,DC차데모+AC3상+DC콤보,20200101045932,20200101053939,0 days 00:40:00,23.40,...,하계로96번길,94-4,주유소,옆,주차장,NaN,NaN,NaN,NaN,NaN
226,진영휴게소(부산방향),1,경상남도,김해시,경상남도 김해시 진영읍 하계로96번길 94-4 주유소 옆 주차장,DC차데모+AC3상+DC콤보,20200101054056,20200101060749,0 days 00:26:47,15.97,...,하계로96번길,94-4,주유소,옆,주차장,NaN,NaN,NaN,NaN,NaN


In [19]:
new_adr_df2[new_adr_df2['2'] == '진영읍'].shape

(33856, 40)

In [24]:
new_adr_df2[(new_adr_df2['충전시간'] != '0') & (new_adr_df2['충전량'] == 0)]

,충전소명,충전기ID,지역,시군구,주소,충전기타입,충전시작일시,충전종료일시,충전시간,충전량,...,3,4,5,6,7,8,9,10,11,12


In [26]:
new_adr_df2[(new_adr_df2['충전시간'] == '0') & (new_adr_df2['충전량'] != 0)].shape

(3192, 40)

In [27]:
new_adr_df2.head()

,충전소명,충전기ID,지역,시군구,주소,충전기타입,충전시작일시,충전종료일시,충전시간,충전량,...,3,4,5,6,7,8,9,10,11,12
0,진영휴게소(부산방향),1,경상남도,김해시,경상남도 김해시 진영읍 하계로96번길 94-4 주유소 옆 주차장,DC차데모+AC3상+DC콤보,20200101000244,20200101004215,0 days 00:39:25,20.22,...,하계로96번길,94-4,주유소,옆,주차장,NaN,NaN,NaN,NaN,NaN
1,민속자연사박물관,1,제주특별자치도,제주시,제주특별자치도 제주시 삼성로 40 민속자연사박물관,DC차데모+AC3상+DC콤보,20200101000745,20200101003703,0 days 00:29:13,16.04,...,40,민속자연사박물관,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,노형동주민센터,1,제주특별자치도,제주시,제주특별자치도 제주시 노형9길 9-4 주차장,DC차데모+AC3상+DC콤보,20200101000748,20200101001659,0 days 00:09:07,5.80,...,9-4,주차장,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,영일대1주차장,1,경상북도,포항시,경상북도 포항시 북구 해안로 82,DC차데모+AC3상+DC콤보,20200101000851,20200101004220,0 days 00:33:29,13.32,...,해안로,82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,38선 휴게소,1,강원도,양양군,강원도 양양군 현북면 동해대로 1242,DC차데모+AC3상+DC콤보,20200101001042,20200101005123,0 days 00:40:41,29.23,...,동해대로,1242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
new_adr_df2.to_numpy()[:5]

array([['진영휴게소(부산방향)', 1, '경상남도', '김해시',
        '경상남도 김해시 진영읍 하계로96번길 94-4  주유소 옆 주차장', 'DC차데모+AC3상+DC콤보',
        20200101000244, 20200101004215, Timedelta('0 days 00:39:25'),
        20.22, 50.0, 2020, 1, 1, 0, 2, 44, '2020-01-01 00:02:44', 2020,
        1, 1, 0, 42, 15, '2020-01-01 00:42:15',
        Timedelta('0 days 00:39:31'), '0 days 00:00:06', '경상남도', '김해시',
        '진영읍', '하계로96번길', '94-4', '주유소', '옆', '주차장', nan, nan, nan, nan,
        nan],
       ['민속자연사박물관', 1, '제주특별자치도', '제주시', '제주특별자치도 제주시 삼성로 40 민속자연사박물관',
        'DC차데모+AC3상+DC콤보', 20200101000745, 20200101003703,
        Timedelta('0 days 00:29:13'), 16.04, 50.0, 2020, 1, 1, 0, 7, 45,
        '2020-01-01 00:07:45', 2020, 1, 1, 0, 37, 3,
        '2020-01-01 00:37:03', Timedelta('0 days 00:29:18'),
        '0 days 00:00:05', '제주특별자치도', '제주시', '삼성로', '40', '민속자연사박물관',
        nan, nan, nan, nan, nan, nan, nan, nan],
       ['노형동주민센터', 1, '제주특별자치도', '제주시', '제주특별자치도 제주시 노형9길 9-4  주차장',
        'DC차데모+AC3상+DC콤보', 2020010100

In [29]:
new_adr_df2[new_adr_df2['2'] == '노형9길']

,충전소명,충전기ID,지역,시군구,주소,충전기타입,충전시작일시,충전종료일시,충전시간,충전량,...,3,4,5,6,7,8,9,10,11,12
2,노형동주민센터,1,제주특별자치도,제주시,제주특별자치도 제주시 노형9길 9-4 주차장,DC차데모+AC3상+DC콤보,20200101000748,20200101001659,0 days 00:09:07,5.80,...,9-4,주차장,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,노형동주민센터,1,제주특별자치도,제주시,제주특별자치도 제주시 노형9길 9-4 주차장,DC차데모+AC3상+DC콤보,20200101020012,20200101021303,0 days 00:12:46,8.47,...,9-4,주차장,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,노형동주민센터,1,제주특별자치도,제주시,제주특별자치도 제주시 노형9길 9-4 주차장,DC차데모+AC3상+DC콤보,20200101040618,20200101043457,0 days 00:28:33,16.21,...,9-4,주차장,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273,노형동주민센터,1,제주특별자치도,제주시,제주특별자치도 제주시 노형9길 9-4 주차장,DC차데모+AC3상+DC콤보,20200101063025,20200101070141,0 days 00:31:10,17.77,...,9-4,주차장,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
393,노형동주민센터,3,제주특별자치도,제주시,제주특별자치도 제주시 노형9길 9-4 주차장,DC차데모+AC3상+DC콤보,20200101081007,20200101084309,0 days 00:33:02,23.14,...,9-4,주차장,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3777078,노형동주민센터,3,제주특별자치도,제주시,제주특별자치도 제주시 노형9길 9-4 주차장,DC차데모+AC3상+DC콤보,20211011152225,20211011153712,0 days 00:14:47,6.85,...,9-4,주차장,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3779180,노형동주민센터,3,제주특별자치도,제주시,제주특별자치도 제주시 노형9길 9-4 주차장,DC차데모+AC3상+DC콤보,20211011185309,20211011190921,0 days 00:16:12,7.53,...,9-4,주차장,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3781894,노형동주민센터,3,제주특별자치도,제주시,제주특별자치도 제주시 노형9길 9-4 주차장,DC차데모+AC3상+DC콤보,20211012085626,20211012092624,0 days 00:29:58,8.72,...,9-4,주차장,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3782574,노형동주민센터,3,제주특별자치도,제주시,제주특별자치도 제주시 노형9길 9-4 주차장,DC차데모+AC3상+DC콤보,20211012102941,20211012104212,0 days 00:12:31,7.82,...,9-4,주차장,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
new_adr_df2[new_adr_df2['2'] == '노형9길']['0'].unique()

array(['제주특별자치도'], dtype=object)

In [32]:
new_adr_df2[new_adr_df2['12'].notnull() == True]['0'].unique()

array(['경기도', '경상북도'], dtype=object)

In [33]:
new_adr_df2[new_adr_df2['12'].notnull() == True]['1'].unique()

array(['이천시', '경산시'], dtype=object)

In [34]:
new_adr_df2[new_adr_df2['12'].notnull() == True]['3'].unique()

array(['덕이로154번길', '경부고속도로'], dtype=object)

In [35]:
new_adr_df2[new_adr_df2['12'].notnull() == True]['4'].unique()

array(['287-76', '105'], dtype=object)

In [31]:
new_adr_df2['12'].unique()

array([nan, '앞)', '주차장'], dtype=object)

### 교체 충전기 수 예측
- 충전기 별 충전량/충전시간 데이터 활용
- 같은 지역내에 동일한 "충전기ID" 존재

In [5]:
new_adr_df2.columns

Index(['충전소명', '충전기ID', '지역', '시군구', '주소', '충전기타입', '충전시작일시', '충전종료일시', '충전시간',
       '충전량', '충전용량', '충전시작_년도', '충전시작_월', '충전시작_일', '충전시작_시', '충전시작_분',
       '충전시작_초', '충전시작일시(변환)', '충전종료_년도', '충전종료_월', '충전종료_일', '충전종료_시',
       '충전종료_분', '충전종료_초', '충전종료일시(변환)', '충전시간(계산)', 'raw값과_계산값_차이', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'],
      dtype='object')

In [6]:
temp = new_adr_df2[['충전기ID', '지역', '시군구', '충전기타입', '충전시간', '충전량']]
temp.head()

,충전기ID,지역,시군구,충전기타입,충전시간,충전량
0,1,경상남도,김해시,DC차데모+AC3상+DC콤보,0 days 00:39:25,20.22000
1,1,제주특별자치도,제주시,DC차데모+AC3상+DC콤보,0 days 00:29:13,16.04000
2,1,제주특별자치도,제주시,DC차데모+AC3상+DC콤보,0 days 00:09:07,5.80000
3,1,경상북도,포항시,DC차데모+AC3상+DC콤보,0 days 00:33:29,13.32000
4,1,강원도,양양군,DC차데모+AC3상+DC콤보,0 days 00:40:41,29.23000


In [7]:
temp['지역'].unique()

array(['경상남도', '제주특별자치도', '경상북도', '강원도', '대전광역시', '충청남도', '충청북도', '부산광역시',
       '경기도', '전라남도', '대구광역시', '광주광역시', '서울특별시', '울산광역시', '전라북도', '인천광역시',
       '세종특별자치시'], dtype=object)

In [8]:
temp['충전기ID'].unique()

array([ 1,  4,  2,  5, 52, 12,  3, 11,  6, 14, 13, 10, 51,  8, 15, 18, 17,
       42, 41, 44, 43, 22, 21, 24, 23, 16,  7, 91, 80, 89, 83, 88, 84, 86,
       85, 82, 87, 45, 46, 47, 81, 25, 26, 92, 28, 27, 93, 94, 97, 95, 98,
       96], dtype=int8)

In [9]:
temp[temp['지역'] == '서울특별시']['충전기ID'].unique()

array([ 1,  2, 11, 52,  6, 12,  3,  4,  5, 13, 14, 22, 21,  8,  7, 45, 41,
       43, 44, 42, 24, 25, 23, 26], dtype=int8)

In [10]:
temp[temp['지역'] == '서울특별시']['충전기ID'].value_counts()

1     404482
2      52729
22     42123
21     37271
12     28079
11     26816
41     11340
45      8639
44      8049
3       7917
4       7459
42      6339
43      6216
23      5414
52      5070
24      5002
5       3903
26      3707
6       3336
8       3249
25      2697
7       2221
13       961
14       214
Name: 충전기ID, dtype: int64

In [11]:
temp[temp['지역'] == '경기도']['충전기ID'].unique()

array([ 1,  5, 12, 11,  2, 52, 51,  3, 14, 13, 21, 22,  8,  4,  7,  6, 23,
       24, 43, 41, 42, 44, 80, 89, 83, 88, 84, 86, 85, 82, 87, 81, 45, 46,
       91, 26, 25, 28, 27, 92, 94, 97, 95, 93, 98, 96], dtype=int8)

In [12]:
seoul = temp[temp['지역'] == '서울특별시']
seoul.head()

,충전기ID,지역,시군구,충전기타입,충전시간,충전량
55,1,서울특별시,양천구,DC차데모+AC3상+DC콤보,0 days 00:35:51,17.36000
72,1,서울특별시,중구,DC차데모+AC3상+DC콤보,0 days 00:11:38,6.16000
74,1,서울특별시,종로구,DC차데모+AC3상+DC콤보,0 days 00:25:34,15.18000
121,1,서울특별시,관악구,DC차데모+AC3상+DC콤보,0 days 00:14:38,6.63000
150,1,서울특별시,관악구,DC차데모+AC3상+DC콤보,0 days 00:05:47,1.88000


In [13]:
seoul['시군구'].unique()

array(['양천구', '중구', '종로구', '관악구', '동작구', '영등포구', '서대문구', '용산구', '강남구',
       '중랑구', '서초구', '강동구', '강북구', '동대문구', '강서구', '마포구', '도봉구', '은평구',
       '송파구', '금천구', '구로구', '노원구', '성북구', '성동구', '광진구'], dtype=object)

In [14]:
seoul[seoul['시군구'] == '영등포구']['충전기ID'].unique()

array([ 1,  2, 52,  6,  4,  5,  3], dtype=int8)

In [15]:
seoul[seoul['시군구'] == '영등포구']['충전기ID'].value_counts()

1     48225
2     11116
52     5070
5      1853
4      1849
6      1391
3      1379
Name: 충전기ID, dtype: int64

In [16]:
seoul[seoul['시군구'] == '송파구']['충전기ID'].unique()

array([ 1,  2, 44, 41, 43, 42, 45, 21, 23, 25, 26, 24, 22], dtype=int8)

##### 특정 지역 충전기 확인
- '강원도 강릉시 성산면 영동고속도로 231 (보광리)'

In [17]:
new_adr_df2[(new_adr_df2['시군구'] == '영등포구') & (new_adr_df2['충전기ID'] == 1)]['주소'].head()

236          서울특별시 영등포구 의사당대로 1 국회도서관 앞
780     서울특별시 영등포구 영중로3길 7  지하1층 여성전용구역
805     서울특별시 영등포구 영중로 125 지상주차장(외부 1F)
1243    서울특별시 영등포구 영중로 125 지상주차장(외부 1F)
1414      서울특별시 영등포구 양산로 111  왼쪽 끝편 주차장
Name: 주소, dtype: object

In [18]:
new_adr_df2[(new_adr_df2['시군구'] == '강릉시') & (new_adr_df2['충전기ID'] == 1)]['주소'].head()

85     강원도 강릉시 성산면 영동고속도로 231 (보광리)
133    강원도 강릉시 성산면 영동고속도로 231 (보광리)
174    강원도 강릉시 성산면 영동고속도로 231 (보광리)
215    강원도 강릉시 성산면 영동고속도로 231 (보광리)
254    강원도 강릉시 성산면 영동고속도로 231 (보광리)
Name: 주소, dtype: object

In [19]:
new_adr_df2[(new_adr_df2['시군구'] == '강릉시') & (new_adr_df2['주소'] == '강원도 강릉시 성산면 영동고속도로 231 (보광리)')].groupby(['충전기ID'])[['충전시간', '충전량']].agg(['sum', 'mean'])

충전시간                                   충전량         
                    sum                      mean         sum     mean
충전기ID                                                                 
1     106 days 01:21:20 0 days 00:27:53.047288661 95304.53000 17.40086
2      66 days 13:39:36 0 days 00:26:23.583700440 62449.30000 17.19419
3      89 days 05:15:44 0 days 00:26:49.299373695 83053.01000 17.33883

In [20]:
new_adr_df2[(new_adr_df2['시군구'] == '강릉시') & (new_adr_df2['주소'] == '강원도 강릉시 성산면 영동고속도로 231 (보광리)')][['충전시간', '충전량']].agg(['sum', 'mean'])

,충전시간,충전량
sum,261 days 20:16:40,240806.84000
mean,0 days 00:27:07.699834520,17.32548


#### 지역, 시군구, 충전기ID 별 통계

In [21]:
# statis = temp.groupby(['지역', '시군구', '충전기ID'])[['충전시간', '충전량']].agg(['sum', 'mean'])
# statis

In [22]:
# statis.to_excel('분석/2020-22년_신주소_지역_시군구_충전기ID_충전시간_충전량_통계.xlsx', encoding='utf-8')

### 특정 지역/기간별 충전량

In [23]:
new_adr_df2.columns

Index(['충전소명', '충전기ID', '지역', '시군구', '주소', '충전기타입', '충전시작일시', '충전종료일시', '충전시간',
       '충전량', '충전용량', '충전시작_년도', '충전시작_월', '충전시작_일', '충전시작_시', '충전시작_분',
       '충전시작_초', '충전시작일시(변환)', '충전종료_년도', '충전종료_월', '충전종료_일', '충전종료_시',
       '충전종료_분', '충전종료_초', '충전종료일시(변환)', '충전시간(계산)', 'raw값과_계산값_차이', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'],
      dtype='object')

In [24]:
temp = new_adr_df2[['지역', '시군구', '충전시작_년도', '충전시작_월', '충전량']]
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7755902 entries, 0 to 7755901
Data columns (total 5 columns):
 #   Column   Dtype  
---  ------   -----  
 0   지역       object 
 1   시군구      object 
 2   충전시작_년도  int16  
 3   충전시작_월   int8   
 4   충전량      float64
dtypes: float64(1), int16(1), int8(1), object(2)
memory usage: 199.7+ MB


In [25]:
statis = temp.groupby(['지역', '시군구', '충전시작_년도', '충전시작_월'])[['충전량']].agg(['sum', 'mean'])
statis

충전량         
                                 sum     mean
지역   시군구 충전시작_년도 충전시작_월                      
강원도  강릉시 2020    1       24097.21000 15.61712
                 2       19731.80000 15.63534
                 3       19599.25000 16.17100
                 4       20738.49000 17.02667
                 5       23248.08000 16.48800
...                              ...      ...
충청북도 충주시 2022    4      108623.48000 19.98592
                 5      115388.01000 19.89105
                 6      106327.68000 20.13019
                 7      139095.34000 21.49518
                 8      199168.47000 23.55908

[7513 rows x 2 columns]

In [27]:
statis.to_excel('분석/2020-22년_신주소_지역_시군구_년도_월_충전량_통계.xlsx', encoding='utf-8')

### 연도/분기/시간대별 충전기 대수
- 예시) 2020년/1분기(1-3월)/00-01시 충전기 대수

In [9]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11613449 entries, 0 to 11613448
Data columns (total 40 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int8           
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기타입         object         
 6   충전시작일시        int64          
 7   충전종료일시        int64          
 8   충전시간          timedelta64[ns]
 9   충전량           float64        
 10  충전용량          object         
 11  충전시작_년도       int16          
 12  충전시작_월        int8           
 13  충전시작_일        int8           
 14  충전시작_시        int8           
 15  충전시작_분        int8           
 16  충전시작_초        int8           
 17  충전시작일시(변환)    object         
 18  충전종료_년도       int16          
 19  충전종료_월        int8           
 20  충전종료_일        int8           
 21  충전종료_시        int8           
 22  충전종료_분        int8           
 23  충전종료_

In [10]:
df2[(df2['충전시작_년도'] == 2020) & 
    (df2['충전시작_월'] >= 1) & (df2['충전시작_월'] <= 3) & (df2['충전시작_일'] == 5) &
    (df2['충전시작_시'] >= 0) & (df2['충전시작_시'] < 1) & (df2['충전소명'] == '내린천휴게소 양양방향')].iloc[:10, :7]

,충전소명,충전기ID,지역,시군구,주소,충전기타입,충전시작일시
7765000,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),DC차데모+AC3상+DC콤보,20200105005728
7883070,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),DC차데모+AC3상+DC콤보,20200305000732


In [11]:
df2[(df2['충전시작_년도'] == 2020) & 
    (df2['충전시작_월'] >= 1) & (df2['충전시작_월'] <= 3) & (df2['충전시작_일'] == 5) &
    (df2['충전시작_시'] >= 0) & (df2['충전시작_시'] < 1) & (df2['충전소명'] == '내린천휴게소 양양방향')]['충전기ID'].unique()

array([1], dtype=int8)

In [12]:
# 시간 오래걸림
    # 53분동안 2020년도 처리 안됨
# num_charger_by_hours = pd.DataFrame(columns=['년도', '월', '일', '시간', '충전기대수'])
# for y in tqdm(range(2020, 2023)):
#     temp_y = df2[df2['충전시작_년도'] == y].reset_index(drop=True)
#     for m in range(1, 11, 3):
#         temp_m = temp_y[(temp_y['충전시작_월'] >= m) & (temp_y['충전시작_월'] <= m+2)].reset_index(drop=True)
#         for d in temp_m['충전시작_일'].unique():
#             temp_d = temp_m[temp_m['충전시작_일'] == d].reset_index(drop=True)
#             for h in range(24):
#                 temp_h = temp_m[(temp_m['충전시작_시'] >= h) & (temp_m['충전시작_시'] < h+1)].reset_index(drop=True)
#                 count = 0
#                 for one in temp_h['충전소명'].unique():
#                     num_charger = len(temp_h[temp_h['충전소명'] == one]['충전기ID'].unique())
#                     count += num_charger
#                 num_charger_by_hours.loc[len(num_charger_by_hours)] = [y, m, d, h, count]

In [32]:
ymdh_col = ['충전시작_년도', '충전시작_월', '충전시작_일', '충전시작_시', '충전소명', '충전기ID']
ymdh = df2.drop_duplicates(ymdh_col, keep='first').reset_index(drop=True)
ymdh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10192033 entries, 0 to 10192032
Data columns (total 40 columns):
 #   Column        Dtype          
---  ------        -----          
 0   충전소명          object         
 1   충전기ID         int8           
 2   지역            object         
 3   시군구           object         
 4   주소            object         
 5   충전기타입         object         
 6   충전시작일시        int64          
 7   충전종료일시        int64          
 8   충전시간          timedelta64[ns]
 9   충전량           float64        
 10  충전용량          object         
 11  충전시작_년도       int16          
 12  충전시작_월        int8           
 13  충전시작_일        int8           
 14  충전시작_시        int8           
 15  충전시작_분        int8           
 16  충전시작_초        int8           
 17  충전시작일시(변환)    object         
 18  충전종료_년도       int16          
 19  충전종료_월        int8           
 20  충전종료_일        int8           
 21  충전종료_시        int8           
 22  충전종료_분        int8           
 23  충전종료_

#### 분기 컬럼 생성

In [33]:
div_month = []
for i in tqdm(ymdh['충전시작_월'].to_list()):
    if (i >= 1) and (i <= 3):
        div_month.append(1)
    elif (i >= 4) and (i <= 6):
        div_month.append(2)
    elif (i >= 7) and (i <= 9):
        div_month.append(3)
    else:
        div_month.append(4)
len(div_month) == 10192033

100%|██████████| 10192033/10192033 [00:04<00:00, 2060263.56it/s]


True

In [34]:
ymdh['충전시작_분기'] = div_month
ymdh.iloc[:3, 6:]

,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,...,4,5,6,7,8,9,10,11,12,충전시작_분기
0,20200101000244,20200101004215,0 days 00:39:25,20.22000,50.00000,2020,1,1,0,2,...,94-4,주유소,옆,주차장,NaN,NaN,NaN,NaN,NaN,1
1,20200101000745,20200101003703,0 days 00:29:13,16.04000,50.00000,2020,1,1,0,7,...,민속자연사박물관,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,20200101000748,20200101001659,0 days 00:09:07,5.80000,50.00000,2020,1,1,0,7,...,주차장,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [35]:
ymdh.iloc[-5:, 6:]

,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,...,4,5,6,7,8,9,10,11,12,충전시작_분기
10192028,20220831235823,20220901001839,0 days 00:20:01,19.22000,급속(200kW동시),2022,8,31,23,58,...,1271-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
10192029,20220831235909,20220901000721,0 days 00:04:53,3.35000,급속(50kW),2022,8,31,23,59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
10192030,20220831235914,20220901002120,0 days 00:22:06,30.14000,급속(100kW멀티),2022,8,31,23,59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
10192031,20220831235931,20220901004251,0 days 00:40:00,21.97000,급속(50kW),2022,8,31,23,59,...,검단4동,주민센터,옆,NaN,NaN,NaN,NaN,NaN,NaN,3
10192032,20220831235934,20220901004057,0 days 00:40:00,27.60000,급속(50kW),2022,8,31,23,59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


In [36]:
group_ymdh = ymdh.groupby(['충전시작_년도', '충전시작_월', '충전시작_일', '충전시작_시'])['충전기ID'].count().rename('충전기_수(대)')
group_ymdh

충전시작_년도  충전시작_월  충전시작_일  충전시작_시
2020     1       1       0           29
                         1           27
                         2           29
                         3           58
                         4           57
                                   ... 
2022     8       31      19        1062
                         20         931
                         21         731
                         22         516
                         23         387
Name: 충전기_수(대), Length: 23376, dtype: int64

In [37]:
group_div = ymdh.groupby(['충전시작_년도', '충전시작_분기', '충전시작_시'])['충전기ID'].count().rename('충전기_수(대)')
group_div

충전시작_년도  충전시작_분기  충전시작_시
2020     1        0          6395
                  1          4578
                  2          3370
                  3          2813
                  4          2871
                            ...  
2022     3        19        63619
                  20        52697
                  21        42154
                  22        30622
                  23        21767
Name: 충전기_수(대), Length: 264, dtype: int64

In [38]:
group_m = ymdh.groupby(['충전시작_월', '충전시작_시'])['충전기ID'].count().rename('충전기_수(대)')
group_m

충전시작_월  충전시작_시
1       0         11834
        1          8327
        2          6264
        3          5245
        4          5920
                  ...  
12      19        38803
        20        32514
        21        26057
        22        19559
        23        13514
Name: 충전기_수(대), Length: 288, dtype: int64

In [39]:
group_h = ymdh.groupby(['충전시작_시'])['충전기ID'].count().rename('충전기_수(대)')
group_h

충전시작_시
0     130386
1      97321
2      74491
3      64911
4      77421
5     127063
6     214952
7     305332
8     424905
9     512267
10    591077
11    664360
12    738851
13    721442
14    738600
15    759144
16    760384
17    715055
18    641412
19    549712
20    459797
21    367166
22    269420
23    186564
Name: 충전기_수(대), dtype: int64

In [40]:
ymdh_file = pd.ExcelWriter('분석/2020-22년_년월일시간별_충전시간.xlsx')
group_h.to_excel(ymdh_file, sheet_name='시간')
group_m.to_excel(ymdh_file, sheet_name='월')
group_div.to_excel(ymdh_file, sheet_name='분기')
group_ymdh.to_excel(ymdh_file, sheet_name='년월일시간')
ymdh_file.save()

### 년도/지역별 충전기 수 통계

In [13]:
df2.iloc[:5, 27:].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       5 non-null      object
 1   1       5 non-null      object
 2   2       5 non-null      object
 3   3       5 non-null      object
 4   4       5 non-null      object
 5   5       1 non-null      object
 6   6       1 non-null      object
 7   7       1 non-null      object
 8   8       0 non-null      object
 9   9       0 non-null      object
 10  10      0 non-null      object
 11  11      0 non-null      object
 12  12      0 non-null      object
dtypes: object(13)
memory usage: 648.0+ bytes


In [14]:
df2['0'].unique()

array(['경상남도', '제주특별자치도', '경상북도', '강원도', '대전광역시', '충청남도', '충청북도', '부산광역시',
       '경기도', '전라남도', '대구광역시', '광주광역시', '서울특별시', '울산광역시', '전라북도', '인천광역시',
       '세종특별자치시'], dtype=object)

In [15]:
# 3.8s
numch_col = ['충전시작_년도', '충전소명', '충전기ID']
numch = df2.drop_duplicates(numch_col, keep='first').reset_index(drop=True)
numch.shape

(17585, 40)

In [16]:
numch_group_ad1 = numch.groupby(['충전시작_년도', '0'])['충전기ID'].count().rename('충전기 수')
numch_group_ad1

충전시작_년도  0      
2020     강원도        428
         경기도        552
         경상남도       407
         경상북도       571
         광주광역시       84
         대구광역시      147
         대전광역시       97
         부산광역시       96
         서울특별시      258
         세종특별자치시     46
         울산광역시      133
         인천광역시      128
         전라남도       419
         전라북도       304
         제주특별자치도    250
         충청남도       382
         충청북도       270
2021     강원도        542
         경기도        763
         경상남도       604
         경상북도       803
         광주광역시      112
         대구광역시      176
         대전광역시      115
         부산광역시       96
         서울특별시      300
         세종특별자치시     49
         울산광역시      158
         인천광역시      163
         전라남도       608
         전라북도       499
         제주특별자치도    249
         충청남도       501
         충청북도       353
2022     강원도        596
         경기도        880
         경상남도       687
         경상북도       903
         광주광역시      133
         대구광역시      187
         대전광역시      134

In [17]:
numch_group_ad2 = numch.groupby(['충전시작_년도', '0', '1'])['충전기ID'].count().rename('충전기 수')
numch_group_ad2

충전시작_년도  0     1  
2020     강원도   강릉시     40
               고성군     14
               동해시     15
               삼척시     43
               속초시     16
                     ... 
2022     충청북도  제천시     33
               증평군      6
               진천군     27
               청주시    107
               충주시     73
Name: 충전기 수, Length: 744, dtype: int64

In [18]:
numch_group_ad3 = numch.groupby(['충전시작_년도', '0', '1', '2'])['충전기ID'].count().rename('충전기 수')
numch_group_ad3

충전시작_년도  0     1    2    
2020     강원도   강릉시  강동면       4
                    강릉대로      1
                    구정면       6
                    남산초교길     2
                    남항진동      2
                             ..
2022     충청북도  충주시  용탄동       1
                    으뜸로       1
                    주덕읍       1
                    중앙탑면     12
                    지현동       1
Name: 충전기 수, Length: 6310, dtype: int64

In [19]:
numch_group_ad4 = numch.groupby(['충전시작_년도', '0', '1', '2', '3'])['충전기ID'].count().rename('충전기 수')
numch_group_ad4

충전시작_년도  0     1    2     3       
2020     강원도   강릉시  강동면   와천로          3
                          정동역길         1
                    강릉대로  33           1
                    구정면   동해고속도로       3
                          회산로          3
                                      ..
2022     충청북도  충주시  으뜸로   21           1
                    주덕읍   신덕로          1
                    중앙탑면  중부내륙고속도로    11
                          탑평리          1
                    지현동   696          1
Name: 충전기 수, Length: 9496, dtype: int64

In [55]:
numch_group_file = pd.ExcelWriter('분석/2020-22년_년도_지역별_충전기_수.xlsx')
numch_group_ad1.to_excel(numch_group_file, sheet_name='지역')
numch_group_ad2.to_excel(numch_group_file, sheet_name='시군구')
numch_group_ad3.to_excel(numch_group_file, sheet_name='읍면동')
numch_group_ad4.to_excel(numch_group_file, sheet_name='리')
numch_group_file.save()

### 연도/지역별 충전기ID 개수

In [24]:
numch_group_chargerID = numch.groupby(['충전시작_년도', '지역', '시군구', '충전기ID'])['주소'].count().rename('충전기ID 수')
numch_group_chargerID

충전시작_년도  지역    시군구  충전기ID
2020     강원도   강릉시  1        15
                    2        11
                    3         5
                    4         1
                    21        1
                             ..
2022     충청북도  충주시  44        2
                    45        1
                    81        1
                    82        1
                    91        1
Name: 충전기ID 수, Length: 5040, dtype: int64

In [28]:
numch_group_chargerID_y = numch.groupby(['지역', '시군구', '충전기ID', '충전시작_년도'])['주소'].count().rename('충전기ID 수')
numch_group_chargerID_y

지역    시군구  충전기ID  충전시작_년도
강원도   강릉시  1      2020       15
                  2021       17
                  2022       20
           2      2020       11
                  2021       11
                             ..
충청북도  충주시  81     2022        1
           82     2022        1
           91     2020        1
                  2021        1
                  2022        1
Name: 충전기ID 수, Length: 5040, dtype: int64

In [49]:
numch_group_chargerID_y_u = numch.groupby(['지역', '시군구', '충전시작_년도'])['충전기ID'].unique().rename('충전기ID 값')
numch_group_chargerID_y_u

지역    시군구  충전시작_년도
강원도   강릉시  2020                [4, 1, 3, 2, 52, 42, 41, 43, 44, 21, 22]
           2021            [3, 1, 52, 2, 4, 42, 43, 41, 22, 21, 44, 45]
           2022               [1, 2, 3, 52, 22, 21, 42, 45, 43, 41, 44]
      고성군  2020                                          [12, 11, 1, 2]
           2021                                          [12, 2, 1, 11]
                                            ...                        
충청북도  청주시  2021          [1, 43, 2, 12, 11, 42, 52, 41, 45, 22, 21, 44]
           2022       [1, 12, 41, 11, 2, 21, 22, 52, 44, 42, 45, 43,...
      충주시  2020          [1, 11, 12, 2, 91, 41, 42, 45, 44, 43, 21, 22]
           2021       [1, 12, 42, 41, 2, 43, 11, 44, 45, 91, 22, 21,...
           2022       [1, 12, 21, 22, 91, 2, 44, 42, 45, 24, 43, 11,...
Name: 충전기ID 값, Length: 744, dtype: object

In [50]:
numch_group_chargerID_y_u.shape

(744,)

In [51]:
numch_group_chargerID_y_u.iloc[0]

array([ 4,  1,  3,  2, 52, 42, 41, 43, 44, 21, 22], dtype=int8)

In [52]:
for i, one in enumerate(numch_group_chargerID_y_u):
    numch_group_chargerID_y_u.iloc[i] = sorted(one)

In [53]:
numch_group_chargerID_y_u

지역    시군구  충전시작_년도
강원도   강릉시  2020                [1, 2, 3, 4, 21, 22, 41, 42, 43, 44, 52]
           2021            [1, 2, 3, 4, 21, 22, 41, 42, 43, 44, 45, 52]
           2022               [1, 2, 3, 21, 22, 41, 42, 43, 44, 45, 52]
      고성군  2020                                          [1, 2, 11, 12]
           2021                                          [1, 2, 11, 12]
                                            ...                        
충청북도  청주시  2021          [1, 2, 11, 12, 21, 22, 41, 42, 43, 44, 45, 52]
           2022       [1, 2, 3, 11, 12, 21, 22, 23, 24, 41, 42, 43, ...
      충주시  2020          [1, 2, 11, 12, 21, 22, 41, 42, 43, 44, 45, 91]
           2021       [1, 2, 11, 12, 21, 22, 23, 24, 41, 42, 43, 44,...
           2022       [1, 2, 3, 4, 11, 12, 21, 22, 23, 24, 41, 42, 4...
Name: 충전기ID 값, Length: 744, dtype: object

In [54]:
numch_group_chargerID_y_nu = numch.groupby(['지역', '시군구', '충전시작_년도'])['충전기ID'].nunique().rename('충전기ID 개수')
numch_group_chargerID_y_nu

지역    시군구  충전시작_년도
강원도   강릉시  2020       11
           2021       12
           2022       11
      고성군  2020        4
           2021        4
                      ..
충청북도  청주시  2021       12
           2022       15
      충주시  2020       12
           2021       14
           2022       18
Name: 충전기ID 개수, Length: 744, dtype: int64

In [55]:
numch_group_chargerID_file = pd.ExcelWriter('분석/2020-22년_년도_지역별_충전기ID_수.xlsx')
numch_group_chargerID.to_excel(numch_group_chargerID_file, sheet_name='sheet1')
numch_group_chargerID_y.to_excel(numch_group_chargerID_file, sheet_name='sheet2')
numch_group_chargerID_y_u.to_excel(numch_group_chargerID_file, sheet_name='sheet3')
numch_group_chargerID_y_nu.to_excel(numch_group_chargerID_file, sheet_name='sheet4')
numch_group_chargerID_file.save()

# 코드 마지막